## clarification

This document is **designed to testing and visualizing** the results. Basiclly all the figures and tables showed in the paper is produced from here(except the method part). Some picture is generated too large and cutting it by code is just an annoying task, so some of the figures shown in the paper is cutted and shape-reorganized. Some figures may added a x-y-z axes by image processing. We don't do any data manipulation to the figures.

The **training code and training data is masked**, while testing code and data is preserved. **Once the paper is received, the full training code and dataset will be all pubished.**

To run the following code, you need a conda environment aligned with mine. Check `environment.yaml` for details. For some pip package that conda don't offers and can't manage, you may need to check `requirements.txt` to install the remaining packages. 
- Note 1: you should always install all the packages first, see`environment.yml`. Some machine may enconter problems with `xformers` package, especially for windows. You can just comment all the xformers in the importing code and don't install this package, we don't use xformers to accelerate the neural network this time(But you need to reinstall all the packages in a new conda env, because a xformers installation faliure sometimes break other packages). 
- Note 2: you may enconter pip warnings even errors, but don't warry it too much as long as the code runs properly.
- Note 3: It will take about 1.5 hours to run the whole code, the code before "diffusion extra test" will takes about a hour
- Note 4: My device is single GPU 2080ti, multi-GPU platform may enconter errors.
- Note 5: after running the code, there will be a "output" directory, which contains all the visualizations. The notebook maybe warns "cannot save due to size too large", please clear all the cell outputs. If you feel too laggy in the notebook, please adjust the visualization functions to do not display. Personally, I use VSCode jupyter extension and do not feel laggy.

The cases numbering in the code is different from the paper. The correspondence is as follows:
| case here | case in paper |
|-----------|---------------|
| case 1    |               |
| case 2    |               |
| case 3    |               |
| case 4    |               |
| case 5    |               |
| case 6    | case 2        |
| case 7    |               |
| case 8    |              
| case 9    | case 1        |
| case 10   | case 3        |
| case 11   | case 4        |
| case 12   | case 5        |

Numerical Simulation time for each case：
1.   99460s
2.   95361s
3.   112859s
4.   172377s
5.   307981s
6.   230962s
7.   228908s
8.   289763s
9.   217596s
10.  218365s
11.  218099s
12.  217031s


## import

In [1]:
from typing import Dict, Optional, Tuple, Callable, Any, List
import os
import shutil
import time
from functools import partial
from enum import Enum
import omegaconf
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import lightning as L
from lightning.pytorch.callbacks import Callback
from lightning.fabric import Fabric
import einops
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
from matplotlib import gridspec
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.subplots as sp
import plotly.io as pio
from rich.console import Console

from my_LDM.utils.dataset_tool.path_tool import get_root_path
from my_LDM.utils.dataset_tool.floris_tool import turbine_layout, realCase_floris_interpolation
from my_LDM.utils.dataset_tool.misc import get_min_max_range, min_max_normalize_tensor, min_max_denormalize_tensor
from my_LDM.utils.effective_U import calculate_effective_3d, calculate_effective_2d
from my_LDM.rectified_flow.oneframe_datamodule_ import Inflow_Area_DataModule
from my_LDM.rectified_flow.lightning_model import Rectified_flow
from my_LDM.oneframe_3d.lightning_model import Oneframe_Area_lightning
# from my_LDM.rectified_flow.lightning_model_copy import Rectified_flow_unet

console = Console()
root_path = get_root_path(root_dir_name="diffusion_wake")

print(f"the SOWFA simulation time of every 10s: {(230962 + 217596 + 218365 + 218099 + 217031) / 5 / 400}")

class Case_idx(Enum):
    case_6 = 0
    case_9 = 1
    case_10 = 2
    case_11 = 3
    case_12 = 4

ModuleNotFoundError: No module named 'my_LDM.utils.dataset_tool.vtk_tool'

In [2]:
def get_turbine_layout_tensor(to_idx: bool = False):
    """从floris_tool导入的布局数据转换为tensor

    Args:
        to_idx (bool, optional): 是否按照20m的网格转换为tensor的索引. Defaults to False.

    Returns:
        list: [[turbine_num, 4], ...]; 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]; 第一个case索引为0
    """

    turbine_layout_tensor = [torch.tensor(case, dtype=torch.float32).transpose(0, 1) for case in turbine_layout[1:]]
    if to_idx:
        for turbine_info in turbine_layout_tensor:
            turbine_info[:, :2] = turbine_info[:, :2] // 20
    return turbine_layout_tensor


## preprocess

为了方便网络传输，原始数据集被截断为仅包含测试集部分，而以下代码是面向原始数据集的形状的，因此使用0填充缺失位置以确保代码正常运行

In [ ]:
for case_num in range(1, 13):  
    print(f"case_num: {case_num}")

    data_slice = np.load(os.path.join(root_path, "dataset_sliced/flowField_full_3d/150_200", f"adm_{case_num}.npy"))
    data_inflow_slice = np.load(os.path.join(root_path, "dataset_sliced/flowField_measured_area_3d/150_200", f"adm_{case_num}.npy"))

    print(f"data_slice.shape: {data_slice.shape}")
    print(f"data_inflow_slice.shape: {data_inflow_slice.shape}")

    # original dataset shape
    target_shape_full = (401, 3, 12, 150, 200)
    target_shape_inflow = (401, 3, 12, 150, 4)

    data_full = np.zeros(target_shape_full, dtype=data_slice.dtype)
    data_inflow = np.zeros(target_shape_inflow, dtype=data_inflow_slice.dtype)

    data_full[-60:] = data_slice
    data_inflow[-200:] = data_inflow_slice

    os.makedirs(os.path.join(root_path, "dataset/flowField_full_3d/150_200"), exist_ok=True)
    os.makedirs(os.path.join(root_path, "dataset/flowField_measured_area_3d/150_200"), exist_ok=True)

    np.save(os.path.join(root_path, "dataset/flowField_full_3d/150_200", f"adm_{case_num}.npy"), data_full)
    np.save(os.path.join(root_path, "dataset/flowField_measured_area_3d/150_200", f"adm_{case_num}.npy"), data_inflow)


# 将adm_0直接从dataset_sliced/flowField_full_3d/150_200复制到dataset/flowField_full_3d/150_200
shutil.copy(os.path.join(root_path,"dataset_sliced/flowField_full_3d/150_200/adm_0.npy"), 
            "dataset/flowField_full_3d/150_200/adm_0.npy",)
shutil.copy(os.path.join(root_path,"dataset_sliced/flowField_measured_area_3d/150_200/adm_0.npy"), 
            "dataset/flowField_measured_area_3d/150_200/adm_0.npy")
shutil.copytree(os.path.join(root_path,"dataset_sliced/Floris_realCase"), 
            "dataset/Floris_realCase",
            dirs_exist_ok=True)
shutil.copytree(os.path.join(root_path,"dataset_sliced/flowField_full"), 
            "dataset/flowField_full",
            dirs_exist_ok=True)



## Visualizing tool funcs

#### 3-D data visual in 3D

In [ ]:
def plot_3d_slices(
    data_tensor: torch.Tensor,
    C_to_view: int,
    x_slices: list[int],
    y_slices: list[int],
    z_slices: list[int],
    title: str = None,
    min_max: Tuple[float, float] = None,
    aspectratio=None,
    scale_factor=20,  # 定义比例因子，20m一个点
    z_offset=20,  # z轴从20m开始
    hide_axes=False,
    figure_height=800,  # 设置默认的图形高度
    initial_camera_eye=dict(x=1.2, y=1.2, z=1.2),  # 默认视角
    save_path=None,  # 新增保存路径参数
):
    """使用 Plotly 绘制 3D 张量的切片，并可选择保存为文件。

    Args:
        data_tensor (torch.Tensor): 存储张量，形状为 [C, Z, H, W]
        C_to_view (int): 要查看的速度分量 (x: 0, y: 1, z: 2)
        x_slices (list[int]): 垂直于x轴的切片，传入整数(作为索引)的列表
        y_slices (list[int]): 垂直于y轴的切片，传入整数(作为索引)的列表
        z_slices (list[int]): 垂直于z轴的切片，传入整数(作为索引)的列表
        aspectratio (dict, optional): 控制 3D 图像的轴比例
        scale_factor (int, optional): 坐标的比例因子，默认 20
        z_offset (int, optional): Z 轴的高度偏移，默认 20
        hide_axes (bool, optional): 是否隐藏坐标轴，默认不隐藏
        figure_height (int, optional): 控制图形的高度，默认 800
        initial_camera_eye (dict, optional): 初始相机视角，默认从对面看
        save_path (str, optional): 若指定路径，则保存图像到该路径，默认不保存。
    """
    # 获取张量的大小
    C, Z, H, W = data_tensor.shape

    # 创建一个空的 Figure
    fig = go.Figure()

    # 计算张量的最小值和最大值，确保所有切片共用相同的colorbar
    if min_max is not None:
        data_min, data_max = min_max
    else:
        data_min = data_tensor[C_to_view].min().item()
        data_max = data_tensor[C_to_view].max().item()

    # 绘制 Z 平面的水平切片
    for z in z_slices:
        z_slice = data_tensor[C_to_view, z, :, :].cpu().numpy()  # 提取 Z 方向的切片
        x_grid, y_grid = np.meshgrid(
            np.linspace(0, W * scale_factor, W), np.linspace(0, H * scale_factor, H)
        )  # 创建 X 和 Y 的网格
        fig.add_trace(
            go.Surface(
                z=np.ones_like(z_slice) * (z + 1) * scale_factor + z_offset,  # 使用 Z 作为高度，并加上偏移
                x=x_grid,  # X 轴坐标
                y=y_grid,  # Y 轴坐标
                surfacecolor=z_slice,  # 使用切片中的数据作为颜色
                colorscale="RdBu_r",  # 使用 RdBu_r 颜色映射
                cmin=data_min,  # 统一的最小值
                cmax=data_max,  # 统一的最大值
                opacity=1 if z != 11 else 0.35,  # 设置透明度
                showscale=False,  # 不显示独立的颜色条
            )
        )

    # 绘制 Y 平面的垂直切片
    for y in y_slices:
        y_slice = data_tensor[C_to_view, :, y, :].cpu().numpy()  # 提取 Y 方向的切片
        x_grid, z_grid = np.meshgrid(
            np.linspace(0, W * scale_factor, W), np.linspace(0, Z * scale_factor + z_offset, Z)
        )  # 创建 X 和 Z 的网格
        fig.add_trace(
            go.Surface(
                z=z_grid,  # Z 方向的高度
                x=x_grid,  # X 轴坐标
                y=np.ones_like(x_grid) * y * scale_factor,  # 将 Y 坐标固定为该 Y 平面
                surfacecolor=y_slice,  # 使用切片中的数据作为颜色
                colorscale="RdBu_r",  # 使用 RdBu_r 颜色映射
                cmin=data_min,  # 统一的最小值
                cmax=data_max,  # 统一的最大值
                opacity=1 if y != 0 else 0.35,  # 设置透明度
                showscale=False,  # 不显示独立的颜色条
            )
        )

    # 绘制 X 平面的垂直切片
    for x in x_slices:
        x_slice = data_tensor[C_to_view, :, :, x].cpu().numpy()  # 提取 X 方向的切片
        y_grid, z_grid = np.meshgrid(
            np.linspace(0, H * scale_factor, H), np.linspace(0, Z * scale_factor + z_offset, Z)
        )  # 创建 Y 和 Z 的网格
        fig.add_trace(
            go.Surface(
                z=z_grid,  # Z 方向的高度
                x=np.ones_like(y_grid) * x * scale_factor,  # 将 X 坐标固定为该 X 平面
                y=y_grid,  # Y 轴坐标
                surfacecolor=x_slice,  # 使用切片中的数据作为颜色
                colorscale="RdBu_r",  # 使用 RdBu_r 颜色映射
                cmin=data_min,  # 统一的最小值
                cmax=data_max,  # 统一的最大值
                opacity=1 if x != 199 else 0.35,  # 设置透明度
                showscale=True,  # 只有最后一个切片显示颜色条
                colorbar=dict(
                    title="Velocity",  # colorbar 标题
                    titleside="right",
                ),
            )
        )

    # 处理坐标轴是否隐藏
    axis_visibility = False if hide_axes else True

    # 更新布局，调整坐标轴比例、可见性和初始视角
    fig.update_layout(
        title=title if title is not None else "3D Slices Visualization of Velocity Field",
        scene=dict(
            xaxis=dict(title="X Axis", visible=axis_visibility),
            yaxis=dict(title="Y Axis", visible=axis_visibility),
            zaxis=dict(title="Z Axis", visible=axis_visibility),
            aspectratio=aspectratio if aspectratio is not None else dict(x=2, y=1.5, z=0.12),  # 调整比例
            camera=dict(eye=initial_camera_eye),  # 设置初始视角
        ),
        height=figure_height,  # 控制图形的高度
        paper_bgcolor="rgba(0,0,0,0)",  # 设置图形背景为透明
        plot_bgcolor="rgba(0,0,0,0)",  # 设置绘图区域背景为透明
    )

    # 如果指定了保存路径，则保存图像
    if save_path:
        # 检查并创建目录
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 保存图像
        fig.write_image(save_path)
        print(f"Figure saved to {save_path}")

    # 展示图像
    fig.show()

# Zs = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
# Z =  [0,  1,  2,  3,  4,   5,   6,   7,   8,   9,   10,  11]
# C =  [0,  3,  6,  9,  12,  15,  18,  21,  24,  27,  30,  33]


# 绘制真实尾流场一帧
wake_data = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_9.npy"))
wake_data = torch.from_numpy(wake_data).float()
wake_data = wake_data[-10]

# 绘制ground truth
plot_3d_slices(
    wake_data,
    C_to_view=0,
    x_slices=[100, 0, 199],
    y_slices=[75, 149, 0],
    z_slices=[0, 6, 11],
    title="Ground Truth",
    min_max=(4, 12),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=True,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/3d_box_wake_field_case1_t=3900.png"  # 保存路径
)



#### 2-D data direct visual

In [5]:
def plot_2d_slice(data, width=800, height=600, title=None, colorbar_thickness=10, colorbar_len=1.25, save_path=None):
    """绘制2d数据，要求传入的数据是两个维度的(即矩阵)

    Args:
        data (torch.Tensor|np.ndarray): 传入的矩阵
        width (int, optional): 绘图设置. Defaults to 800.
        height (int, optional): 绘图设置. Defaults to 600.
        title (str, optional): 图像标题. Defaults to None.
        colorbar_thickness (int, optional): 设置颜色条宽度. Defaults to 10.
        colorbar_len (float, optional): 设置颜色条长度. Defaults to 1.25.
        save_path (str, optional): 如果非 None，则保存图像到指定路径. Defaults to None.
    """
    # 如果输入是 torch.Tensor，则转换为 numpy 数组
    if isinstance(data, torch.Tensor):
        data = data.cpu().numpy()

    # 创建图像
    fig = px.imshow(
        data[:, :],
        color_continuous_scale="RdBu_r",
    )

    # 更新布局
    fig.update_layout(
        title=title,  # 设置标题
        title_x=0.5,  # 设置标题居中
        title_y=0.95,  # 设置标题稍微靠近顶部
        width=width,  # 设置图像宽度
        height=height,  # 设置图像高度
        coloraxis_colorbar=dict(
            thickness=colorbar_thickness,  # 设置颜色条的宽度
            len=colorbar_len,  # 设置颜色条的长度 (0.0-1.0)
            y=0.5,  # 设置颜色条的垂直位置
            yanchor="middle",  # 颜色条的锚点
        ),
    )

    # 如果指定了保存路径，则保存图像
    if save_path:
        # 检查并创建保存路径的目录
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 保存图片
        fig.write_image(save_path)
        print(f"Figure saved to {save_path}")

    # 展示图像
    fig.show()

#### 3-D data multi-height slices

In [6]:
# # Zs = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
# # Z =  [0,  1,  2,  3,  4,   5,   6,   7,   8,   9,   10,  11]
# # C =  [0,  3,  6,  9,  12,  15,  18,  21,  24,  27,  30,  33]


def plot_channels_for_test(
    tensor_pred: torch.Tensor,
    tensor_target: torch.Tensor,
    channel: int,
    channel_name="X",
    vmin=-3,
    vmax=11,
    pred_title="Pred",
    target_title="Target",
    grid_size=(3000, 4000),  # 物理尺寸 (单位：米)
    save_path=None,  # 新增保存路径参数
) -> plt.Figure:
    """
    Plot the prediction, target, and difference of a given channel from 3D tensors.

    tensor: [C=3, Z=12, H=150, W=200]

    Z = [0, 1, 2, 3, 4,  5,  6,  7,  8,  9,  10, 11]
    Z = [20,40,60,80,100,120,140,160,180,200,220,240] (单位：米)
    """
    # 将张量转换为 NumPy 数组
    tensor_pred = tensor_pred.detach().cpu().numpy()
    tensor_target = tensor_target.detach().cpu().numpy()
    tensor_diff = np.abs(tensor_pred - tensor_target)

    # 获取网格的大小和物理尺寸
    H, W = tensor_pred.shape[2:]  # 网格尺寸
    x_extent = np.linspace(0, grid_size[0], H)  # 横向物理坐标 (0 到 3000米)
    y_extent = np.linspace(0, grid_size[1], W)  # 纵向物理坐标 (0 到 4000米)
    extent = [y_extent[0], y_extent[-1], x_extent[-1], x_extent[0]]  # extent for imshow

    # 3 行（预测、目标、差值），6 列（6 个高度）
    fig, axes = plt.subplots(3, 6, figsize=(25, 9))
    # 调整子图间距
    fig.subplots_adjust(hspace=0.1, wspace=-0.18)

    selected_Zs = [1, 3, 4, 6, 8, 10]  # 选定的 Z 层
    for i, z in enumerate(selected_Zs):
        Z = 20 + 20 * z  # Z 的物理高度，单位为米

        # 绘制预测
        im1 = axes[0, i].imshow(
            tensor_pred[channel, z], 
            cmap="RdBu_r", 
            vmin=vmin, 
            vmax=vmax, 
            extent=extent
        )

        # 绘制目标
        im2 = axes[1, i].imshow(
            tensor_target[channel, z], 
            cmap="RdBu_r", 
            vmin=vmin, 
            vmax=vmax, 
            extent=extent
        )

        # 绘制差值
        im3 = axes[2, i].imshow(
            tensor_diff[channel, z], 
            cmap="Reds", 
            vmin=0, 
            vmax=vmax, 
            extent=extent
        )

        # 隐藏坐标轴刻度和标签，仅保留框
        for row in range(3):
            axes[row, i].set_xticks([])
            axes[row, i].set_yticks([])

    # 添加列标题（Z 值）
    for i, z in enumerate(selected_Zs):
        Z = 20 + 20 * z
        # 获取每列子图的中心位置
        col_center = axes[0, i].get_position().bounds[0] + axes[0, i].get_position().bounds[2] / 2
        fig.text(
            col_center,  # 标题的水平位置
            0.9,  # 标题的垂直位置，比之前的 0.97 更靠近子图
            f"Z={Z}m", 
            ha="center", 
            va="center", 
            fontsize=16,  # 字体大小
        )

    # 添加行标题（Pred, Target, Diff）
    row_titles = [pred_title, target_title, "Diff"]
    for i, title in enumerate(row_titles):
        # 获取每行子图的中心位置
        row_center = axes[i, 0].get_position().bounds[1] + axes[i, 0].get_position().bounds[3] / 2
        fig.text(
            0.125,  # 标题的水平位置（靠左）
            row_center,  # 标题的垂直位置
            title, 
            ha="center", 
            va="center", 
            fontsize=16,  # 字体大小
            rotation=0  # 水平显示
        )

    # 添加预测和目标的颜色条
    cbar_ax1 = fig.add_axes([0.892, 0.375, 0.01, 0.505])
    fig.colorbar(im1, cax=cbar_ax1, label=f"{channel_name} Value")

    # 添加差值的颜色条
    cbar_ax2 = fig.add_axes([0.892, 0.11, 0.01, 0.24])
    fig.colorbar(im3, cax=cbar_ax2, label="Difference")

    # 如果指定了保存路径，则保存图像
    if save_path:
        # 检查并创建保存路径的目录
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 保存图片
        fig.savefig(save_path, bbox_inches="tight")
        print(f"Figure saved to {save_path}")

    return fig


# # Assuming `data_full` is already loaded and converted to a torch.Tensor as before
# data_full = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_1.npy"))
# data_full = torch.from_numpy(data_full).float()

# # 绘制 X 通道的图像（vmin=1, vmax=11）
# fig_x = plot_channels_for_test(data_full[53], data_full[54], channel=0, vmin=1, vmax=11, channel_name="X")
# # console.print(fig_x)
# fig_x.show()

# # 绘制 Y 通道的图像（vmin=-3, vmax=3）
# fig_y = plot_channels_for_test(data_full[53], data_full[54], channel=1, vmin=-3, vmax=3, channel_name="Y")
# fig_y.show()

# # 绘制 Z 通道的图像（vmin=-3, vmax=3）
# fig_z = plot_channels_for_test(data_full[53], data_full[54], channel=2, vmin=-3, vmax=3, channel_name="Z")
# fig_z.show()

In [7]:
# # 废弃
# # 每个子图带坐标轴，各自带有标题

# def plot_channels_for_test(
#     tensor_pred: torch.Tensor,
#     tensor_target: torch.Tensor,
#     channel: int,
#     channel_name="X",
#     vmin=-3,
#     vmax=11,
#     pred_title="Pred",
#     target_title="Target",
#     grid_size=(3000, 4000),  # 物理尺寸 (单位：米)
# ) -> plt.Figure:
#     """
#     Plot the prediction, target, and difference of a given channel from 3D tensors.

#     tensor: [C=3, Z=12, H=150, W=200]

#     Z = [0, 1, 2, 3, 4,  5,  6,  7,  8,  9,  10, 11]
#     Z = [20,40,60,80,100,120,140,160,180,200,220,240] (单位：米)
#     """
#     # 将张量转换为 NumPy 数组
#     tensor_pred = tensor_pred.detach().cpu().numpy()
#     tensor_target = tensor_target.detach().cpu().numpy()
#     tensor_diff = np.abs(tensor_pred - tensor_target)

#     # 获取网格的大小和物理尺寸
#     H, W = tensor_pred.shape[2:]  # 网格尺寸
#     x_extent = np.linspace(0, grid_size[0], H)  # 横向物理坐标 (0 到 3000米)
#     y_extent = np.linspace(0, grid_size[1], W)  # 纵向物理坐标 (0 到 4000米)
#     extent = [y_extent[0], y_extent[-1], x_extent[-1], x_extent[0]]  # extent for imshow

#     # 3 行（预测、目标、差值），6 列（6 个高度）
#     fig, axes = plt.subplots(3, 6, figsize=(25, 9))
#     selected_Zs = [1, 3, 4, 6, 8, 10]  # 选定的 Z 层
#     for i, z in enumerate(selected_Zs):
#         Z = 20 + 20 * z  # Z 的物理高度，单位为米

#         # 绘制预测
#         im1 = axes[0, i].imshow(
#             tensor_pred[channel, z], 
#             cmap="RdBu_r", 
#             vmin=vmin, 
#             vmax=vmax, 
#             extent=extent
#         )
#         axes[0, i].set_title(f"{pred_title}: {channel_name} Z={Z}m")
#         axes[0, i].set_xlabel("Y (m)")
#         axes[0, i].set_ylabel("X (m)")

#         # 绘制目标
#         im2 = axes[1, i].imshow(
#             tensor_target[channel, z], 
#             cmap="RdBu_r", 
#             vmin=vmin, 
#             vmax=vmax, 
#             extent=extent
#         )
#         axes[1, i].set_title(f"{target_title}: {channel_name} Z={Z}m")
#         axes[1, i].set_xlabel("Y (m)")
#         axes[1, i].set_ylabel("X (m)")

#         # 绘制差值
#         im3 = axes[2, i].imshow(
#             tensor_diff[channel, z], 
#             cmap="Reds", 
#             vmin=0, 
#             vmax=vmax, 
#             extent=extent
#         )
#         axes[2, i].set_title(f"Diff: {channel_name} Z={Z}m")
#         axes[2, i].set_xlabel("Y (m)")
#         axes[2, i].set_ylabel("X (m)")

#     # 调整子图布局以腾出空间放置颜色条
#     fig.subplots_adjust(hspace=0.4, wspace=0.3)

#     # 添加预测和目标的颜色条
#     cbar_ax1 = fig.add_axes([0.92, 0.4, 0.01, 0.475])
#     fig.colorbar(im1, cax=cbar_ax1, label=f"{channel_name} Value")

#     # 添加差值的颜色条
#     cbar_ax2 = fig.add_axes([0.92, 0.11, 0.01, 0.2])
#     fig.colorbar(im3, cax=cbar_ax2, label="Difference")

#     return fig

In [8]:
# # 废弃
# # 每个子图的坐标轴是根据tensor idx的

# def plot_channels_for_test(
#     tensor_pred: torch.Tensor,
#     tensor_target: torch.Tensor,
#     channel: int,
#     channel_name="X",
#     vmin=-3,
#     vmax=11,
#     pred_title="Pred",
#     target_title="Target",
# ) -> plt.Figure:
#     """
#     Plot the prediction, target, and difference of a given channel from 3D tensors.

#     tensor: [C=3, Z=12, H, W]

#     Z = [0, 1, 2, 3, 4,  5,  6,  7,  8,  9,  10, 11]
#     Z = [20,40,60,80,100,120,140,160,180,200,220,240]
#     """
#     tensor_pred = tensor_pred.detach().cpu().numpy()
#     tensor_target = tensor_target.detach().cpu().numpy()
#     tensor_diff = np.abs(tensor_pred - tensor_target)

#     # 3 rows: prediction, target, difference; 6 columns: 6 heights
#     fig, axes = plt.subplots(3, 6, figsize=(18, 8))
#     selected_Zs = [1, 3, 4, 6, 8, 10]
#     for i, z in enumerate(selected_Zs):
#         Z = 20 + 20 * z

#         # Plot prediction
#         im1 = axes[0, i].imshow(tensor_pred[channel, z], cmap="RdBu_r", vmin=vmin, vmax=vmax)
#         axes[0, i].set_title(f"{pred_title}: {channel_name} Z={Z}")

#         # Plot target
#         im2 = axes[1, i].imshow(tensor_target[channel, z], cmap="RdBu_r", vmin=vmin, vmax=vmax)
#         axes[1, i].set_title(f"{target_title}: {channel_name} Z={Z}")

#         # Plot difference
#         im3 = axes[2, i].imshow(tensor_diff[channel, z], cmap="Reds", vmin=0, vmax=vmax)
#         axes[2, i].set_title(f"Diff: {channel_name} Z={Z}")

#     # Adjust subplot layout to make room for the colorbar
#     fig.subplots_adjust(right=0.9)

#     # Add a colorbar for prediction and target
#     cbar_ax1 = fig.add_axes([0.92, 0.4, 0.02, 0.45])
#     fig.colorbar(im1, cax=cbar_ax1, label=f"{channel_name} Value")

#     # Add a colorbar for the difference
#     cbar_ax2 = fig.add_axes([0.92, 0.12, 0.02, 0.2])
#     fig.colorbar(im3, cax=cbar_ax2, label="Difference")

#     return fig

#### Speed profiles

In [9]:
# # Zs = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
# # Z =  [0,  1,  2,  3,  4,   5,   6,   7,   8,   9,   10,  11]
# # C =  [0,  3,  6,  9,  12,  15,  18,  21,  24,  27,  30,  33]


def get_hub_height_slices(data_tensor: torch.Tensor):
    """get the hub height plane

    data_tensor: [C, Z, H, W]
    """
    # get 80m and 100m
    data_tensor = data_tensor[:, 3:5]
    # the 80m and 100m mean is equivalent doing linear interpolation
    return data_tensor.mean(dim=1)

def visualize_speed_profile(
    data_tensors_list: List[torch.Tensor],
    x_0D_coord,
    turbines_info: torch.Tensor,
    plot_y_Range: Tuple[int, int],
    plot_Ds_max: int = 20,
    plot_neg_D_max_limit: int = 2,
    Diameter=126,
    line_labels: List[str] = None,
    line_styles: List[str] = None,
    line_colors: List[str] = None,
    line_widths: List[float] = None,
    max_subplots_per_row: int = 11,  # Maximum number of subplots per row
    subplots_aspect_ratio: float = 0.25,  # Aspect ratio of each subplot (width/height)
    save_path: str = None  # 新增保存路径参数
):
    """Plot the speed profile with customizable labels, styles, colors, and widths.

    Args:
        data_tensors_list (List[torch.Tensor]): List of [C, H, W], each a plane of data
        x_0D_coord: the point set to 0D position
        turbines_info (torch.Tensor): [N, 4], each row is [x, y, yaw, is_wake_affected]
        plot_y_Range (Tuple): the range of y to plot, [start_coord, end_coord]
        Diameter (int, optional): turbine swept diameter. Defaults to 126.
        line_labels (List[str], optional): List of labels for each dataset line. Defaults to "Dataset {i}".
        line_styles (List[str], optional): List of line styles (e.g., 'solid', 'dashed', 'dotted').
        line_colors (List[str], optional): List of colors for each dataset line.
        line_widths (List[float], optional): List of line widths for each dataset line.
        max_subplots_per_row (int, optional): Maximum number of subplots per row. Defaults to 12.
        subplots_aspect_ratio (float, optional): Aspect ratio of each subplot (width/height). Defaults to 0.25.
        save_path (str, optional): Path to save the generated plot. If None, the plot is not saved.
    """
    # Ensure all tensors in the list have the same shape
    num_tensors = len(data_tensors_list)
    assert all(data.shape == data_tensors_list[0].shape for data in data_tensors_list), "All tensors must have the same shape."

    # Default line labels if not provided
    if line_labels is None:
        line_labels = [f"Dataset {i+1}" for i in range(num_tensors)]
    
    # Default line styles if not provided
    if line_styles is None:
        line_styles = ['solid'] * num_tensors  # Default to solid lines for all datasets
    
    # Default line colors if not provided
    if line_colors is None:
        line_colors = [plt.cm.viridis(j / num_tensors) for j in range(num_tensors)]  # Assign colors from viridis colormap
    
    # Default line widths if not provided
    if line_widths is None:
        line_widths = [1.5] * num_tensors  # Default to width 1.5 for all lines
    
    # Check that the number of labels, styles, colors, and widths match the number of datasets
    assert len(line_labels) == num_tensors, "Number of labels must match the number of datasets."
    assert len(line_styles) == num_tensors, "Number of styles must match the number of datasets."
    assert len(line_colors) == num_tensors, "Number of colors must match the number of datasets."
    assert len(line_widths) == num_tensors, "Number of widths must match the number of datasets."
    
    # Map human-readable style names to matplotlib line styles
    style_map = {
        'solid': '-',
        'dashed': '--',
        'dotted': ':',
        'dashdot': '-.'
    }
    
    # Convert the provided line styles to matplotlib-compatible styles
    line_styles = [style_map.get(style, '-') for style in line_styles]  # Default to solid if not found
    
    # Extract the data from the first tensor to use it as reference
    data_shape = data_tensors_list[0].cpu().numpy().shape
    x_coord = x_0D_coord
    y_coords = [turbine[1] for turbine in turbines_info]  # Extract y coordinates of turbines

    # Divide field by x location and Diameter
    x_D_left_coord = [x_coord - Diameter * i for i in range(1, int(x_coord // Diameter) + 1)][::-1]
    if len(x_D_left_coord) > plot_neg_D_max_limit:
        x_D_left_coord = x_D_left_coord[-plot_neg_D_max_limit:]

    # Include D=0 in the right side
    x_D_right_coord = [x_coord + Diameter * i for i in range(0, int(4000 - x_coord) // Diameter + 1)]
    x_D_coord = x_D_left_coord + x_D_right_coord

    # Convert coord to index
    x_D_idxs = [int(D // 20) for D in x_D_coord]

    # Limit the number of profiles for clarity (also limit the subplot number)
    if len(x_D_idxs) > plot_Ds_max:
        x_D_idxs = x_D_idxs[:plot_Ds_max]

    # Generate y_values for the entire y-range, not just plot_y_Range
    y_values_full = np.arange(0, data_shape[1] * 20, 20)  # Full y range in meters

    # Determine the number of rows and columns needed for subplots
    num_plots = len(x_D_idxs)
    num_cols = min(max_subplots_per_row, num_plots)  # Columns are capped at max_subplots_per_row
    num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate rows needed

    # Dynamically calculate the figsize based on the number of rows and columns
    subplot_height = 4  # Height of each subplot (in inches)
    subplot_width = subplot_height * subplots_aspect_ratio  # Width of each subplot (in inches)
    figsize = (num_cols * subplot_width + 2, num_rows * subplot_height)  # Added 2 inches for legend space

    # Create subplots with the calculated number of rows and columns
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize, sharey=True)
    if num_rows * num_cols == 1:
        axes = [axes]
    else:
        axes = axes.flatten()  # Flatten the axes array for easy indexing

    # Plot each profile in a subplot
    for i in range(num_plots):
        ax = axes[i]
        for j, data_tensor in enumerate(data_tensors_list):
            # Extract the speed profile along the y-axis for each x location
            data = data_tensor.cpu().numpy()
            speed_profile = data[0, :, x_D_idxs[i]]  # Extract profile for this specific x at index i
            ax.plot(
                speed_profile, 
                y_values_full, 
                color=line_colors[j],  # Use custom color
                label=line_labels[j] if i == 0 else "",  # Label only once for legend
                linestyle=line_styles[j],  # Use custom line style
                linewidth=line_widths[j]  # Use custom line width
            )

        # Draw horizontal lines for each turbine's y-location
        for y in y_coords:
            if plot_y_Range[0] <= y <= plot_y_Range[1]:  # Only plot y-lines within the range
                ax.axhline(y, color="#5becca", linestyle="dotted", linewidth=0.5)  # Plot each turbine's y position

        # Labeling each subplot
        D_label = (x_D_coord[i] - x_coord) // Diameter  # Label positions in terms of D
        ax.set_title(f"{int(D_label)}D")

        # Set axis labels for the first and last subplot
        if i % num_cols == 0:
            ax.set_ylabel("Y [m]")
        ax.set_xlabel("U [m/s]")
        ax.set_xlim(2, 10)  # Adjust speed range (U) limits if necessary

        # Set the y-axis limits based on the plot_y_Range, and remove margins
        ax.set_ylim(plot_y_Range)
        ax.margins(0)  # Remove default margins at the top and bottom of the plot

    # Hide any unused subplots
    for i in range(num_plots, len(axes)):
        fig.delaxes(axes[i])

    # Adjust spacing between subplots
    plt.subplots_adjust(top=0.95, bottom=0.05, left=0.07, right=0.85, wspace=0.3, hspace=0.3)  # Reduced hspace

    # Position the legend next to the last subplot in the bottom row
    if num_plots > 0:
        last_ax = axes[num_plots - 1]
        # Get the position of the last subplot
        last_pos = last_ax.get_position()
        # Place the legend to the right of the last subplot
        legend_x = last_pos.x1 + 0.01  # Slightly offset to the right
        legend_y = last_pos.y0 + (last_pos.y1 - last_pos.y0) / 2  # Center vertically
        fig.legend(
            labels=line_labels,  # Use the provided labels
            loc='center left',
            bbox_to_anchor=(legend_x, legend_y),  # Position it next to the last subplot
            bbox_transform=fig.transFigure,  # Use figure coordinates
            frameon=False,
            fontsize=12,
        )

    # Save the figure if a save path is provided
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure directory exists
        fig.savefig(save_path, bbox_inches="tight", dpi=300)  # Save the figure
        print(f"Figure saved to {save_path}")

    plt.show()

# # -------------------- test with real data -------------------- #
# # case 9 as ground_truth
# data_full_9 = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_9.npy"))
# data_full_9 = torch.from_numpy(data_full_9).float()
# data_9 = data_full_9[-40]
# hub_height_slices_9 = get_hub_height_slices(data_9)
# # case 11 as pred
# data_full_11 = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_11.npy"))
# data_full_11 = torch.from_numpy(data_full_11).float()
# data_11 = data_full_11[-40]
# hub_height_slices_11 = get_hub_height_slices(data_11)
# # case 6 as frozen flow
# data_full_6 = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_6.npy"))
# data_full_6 = torch.from_numpy(data_full_6).float()
# data_6 = data_full_6[-40]
# hub_height_slices_6 = get_hub_height_slices(data_6)

# turbine_layout_tensor_list = get_turbine_layout_tensor()
# turbine_infos = turbine_layout_tensor_list[9 - 1]

# # Example usage with custom labels, styles, colors, and widths
# visualize_speed_profile(
#     [hub_height_slices_9, hub_height_slices_11, hub_height_slices_6],
#     x_0D_coord=turbine_infos[0, 0].item(),  # Set 0D at x=400
#     turbines_info=turbine_infos,  # Turbine info
#     plot_y_Range=(600, 2200),  # Plot y range from 600 to 2200 meters
#     Diameter=126,
#     line_labels=["Ground Truth", "Prediction", "Frozen Hypo"],  # Custom labels
#     line_styles=["solid", "dashed", "dashed"],  # Custom line styles (solid, dashed, dotted)
#     line_colors=["black", "#0061ff", "#ff0059"],  # Custom colors
#     line_widths=[2.5, 1.5, 1.5]  # Custom line widths
# )


## View all cases setting

In [ ]:
modes = ["sowfa", "floris"]

for mode in modes:
    save_path = f"./output/case_setting_{mode}.png"  # 如果不需要保存图片，将此变量设为 None

    data_to_visualize_list = []

    # -------------------- use sowfa simulation ------------------- #
    if mode == "sowfa":
        # 加载数据并处理
        # 注意我们数据集编号与论文编号不同，因此这里需要自己调整顺序
        for i in [9, 6, 10, 11, 12, 1, 2, 3, 4, 5, 7, 8]:
            data_full = torch.from_numpy(np.load(os.path.join(root_path, f"dataset/flowField_full_3d/150_200/adm_{i}.npy"))).float()
            # 提取最后一个时间步（[-80]）3-5维数据的均值
            data_to_visualize_list.append(get_hub_height_slices(data_full[-50]))
            print(f"adm_{i} data.shape: {data_to_visualize_list[-1].shape}")  # [3, 150, 200]

    # ------------------------- use floris ------------------------ #
    if mode == "floris":
        for i in [9, 6, 10, 11, 12, 1, 2, 3, 4, 5, 7, 8]:
            data_floris = torch.from_numpy(np.load(os.path.join(root_path, f"dataset/Floris_realCase/150_200/adm_{i}.npy"))).float()
            data_to_visualize_list.append(data_floris)
            print(f"adm_{i} data.shape: {data_floris.shape}")  # [2, 150, 200]

    # 创建子图
    fig, axes = plt.subplots(3, 4, figsize=(18, 11))  # 3行4列的子图布局

    # 绘制每个子图
    for i, ax in enumerate(axes.flat):  # 使用 axes.flat 将子图数组展平，方便迭代
        im = ax.imshow(data_to_visualize_list[i][0], cmap="RdBu_r")  # 显示第一个通道的数据
        ax.set_title(f"Case {i+1}", fontsize=16)  # 设置标题，调整字体大小为 16

        # 设置刻度为空
        ax.set_xticks([])  # 清除 x 轴刻度
        ax.set_yticks([])  # 清除 y 轴刻度

    # 显示图像
    plt.tight_layout()  # 自动调整子图间距
    plt.show()

    # 如果 save_path 不为 None，则保存图片
    if save_path:
        # 检查路径是否存在，如果不存在则创建
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 保存图片
        fig.savefig(save_path, bbox_inches="tight", dpi=300)  # 使用高分辨率保存
        print(f"Figure saved to {save_path}")

## Frozen Flow Hypo


In [ ]:
# ------------------------- load data ------------------------- #
data_inflow = np.load(os.path.join(root_path, "dataset/flowField_measured_area_3d/150_200/adm_0.npy"))
data_inflow = torch.from_numpy(data_inflow).float()
print(f"data_inflow.shape: {data_inflow.shape}")
# [timestep=4001, channel=3, z=12, y=150, x=4]

data_full = np.load(os.path.join(root_path, "dataset/flowField_full_3d/150_200/adm_0.npy"))
data_full = torch.from_numpy(data_full).float()
print(f"data_full.shape: {data_full.shape}")
# [timestep=4001, channel=3, z=12, y=150, x=200]

# ------------------------- frozen hypo ------------------------- #
SELECT_RANGE = 50
start_timestep = 0

frozen_hypo_result = data_inflow[start_timestep : start_timestep + SELECT_RANGE]
frozen_hypo_result = torch.flip(frozen_hypo_result, dims=[0])
frozen_hypo_result = einops.rearrange(frozen_hypo_result, "T C Z H W -> C Z H (T W)")
# frozen_hypo_result = einops.rearrange(frozen_hypo_result, "C Z H W -> (C Z) H W")
print(f"frozen_hypo_result.shape: {frozen_hypo_result.shape}")
# [C=3, Z=12, H=150, W=200]

# ------------------------ ground truth ----------------------- #
real_result = data_full[start_timestep + SELECT_RANGE - 1]
# real_result = einops.rearrange(real_result, "C Z H W -> (C Z) H W")
print(f"real_result.shape: {real_result.shape}")
# [C=3, Z=12, H=150, W=200]

# ------------------------- difference ------------------------ #
diff_real_hypo = real_result - frozen_hypo_result
print(f"total of diff_real_hypo: {diff_real_hypo.abs().mean()}")

### visualization

#### 3-D slices

##### without semitransparent effect

In [ ]:
# 绘制 frozen hypo
plot_3d_slices(
    frozen_hypo_result,
    C_to_view=0,
    x_slices=[100, 0],
    y_slices=[75, 149],
    z_slices=[0, 6],
    title="Frozen Hypothesis",
    min_max=(4, 12),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=False,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_t=500_pred.png"  # 保存路径
)

# 绘制ground truth
plot_3d_slices(
    real_result,
    C_to_view=0,
    x_slices=[100, 0],
    y_slices=[75, 149],
    z_slices=[0, 6],
    title="Ground Truth",
    min_max=(4, 12),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=False,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_t=500_real.png"  # 保存路径
)

# 绘制 real-frozen diff
plot_3d_slices(
    diff_real_hypo,
    C_to_view=0,
    x_slices=[100, 0],
    y_slices=[75, 149],
    z_slices=[0, 6],
    title="Real - Frozen Difference",
    min_max=(-4, 4),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=False,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_t=500_diff.png"  # 保存路径
)



##### semitransparent effect

In [ ]:
# Zs = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
# Z =  [0,  1,  2,  3,  4,   5,   6,   7,   8,   9,   10,  11]
# C =  [0,  3,  6,  9,  12,  15,  18,  21,  24,  27,  30,  33]


# 绘制 frozen hypo
plot_3d_slices(
    frozen_hypo_result,
    C_to_view=0,
    x_slices=[100, 0, 199],
    y_slices=[75, 149, 0],
    z_slices=[0, 6, 11],
    title="Frozen Hypothesis",
    min_max=(4, 12),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=True,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_box_t=500_pred.png"  # 保存路径
)

# 绘制ground truth
plot_3d_slices(
    real_result,
    C_to_view=0,
    x_slices=[100, 0, 199],
    y_slices=[75, 149, 0],
    z_slices=[0, 6, 11],
    title="Ground Truth",
    min_max=(4, 12),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=True,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_box_t=500_real.png"  # 保存路径
)

# 绘制 real-frozen diff
plot_3d_slices(
    diff_real_hypo,
    C_to_view=0,
    x_slices=[100, 0, 199],
    y_slices=[75, 149, 0],
    z_slices=[0, 6, 11],
    title="Real - Frozen Difference",
    min_max=(-4, 4),
    aspectratio=dict(x=2, y=1.5, z=0.35),
    scale_factor=20,  # 20m一个点
    z_offset=20,  # Z轴从20m开始
    hide_axes=True,  # 隐藏坐标轴
    figure_height=750,  # 图形高度
    initial_camera_eye=dict(x=1.8, y=-1.8, z=0.75),  # 从正对面看
    save_path="./output/frozen_hypo/3d_box_t=500_diff.png"  # 保存路径
)

##### multi-height slicing

In [ ]:
# 绘制 X 通道的图像（vmin=1, vmax=11）
fig_x = plot_channels_for_test(
    frozen_hypo_result,
    real_result,
    channel=0,
    vmin=1,
    vmax=11,
    channel_name="X",
    pred_title="Frozen\nhypo",
    target_title="Real",
    save_path="./output/frozen_hypo/slices_t=500_x.png"  # 保存路径
)
fig_x.show()

# 绘制 Y 通道的图像（vmin=-3, vmax=3）
fig_y = plot_channels_for_test(
    frozen_hypo_result,
    real_result,
    channel=1,
    vmin=-3,
    vmax=3,
    channel_name="Y",
    pred_title="Frozen\nhypo",
    target_title="Real",
    save_path="./output/frozen_hypo/slices_t=500_y.png"  # 保存路径
)
fig_y.show()

# 绘制 Z 通道的图像（vmin=-3, vmax=3）
fig_z = plot_channels_for_test(
    frozen_hypo_result,
    real_result,
    channel=2,
    vmin=-3,
    vmax=3,
    channel_name="Z",
    pred_title="Frozen\nhypo",
    target_title="Real",
    save_path="./output/frozen_hypo/slices_t=500_z.png"  # 保存路径
)
fig_z.show()

#### speed profile

In [ ]:
# ------------------ hub height speed profile ----------------- #

hub_height_slices_pred = get_hub_height_slices(frozen_hypo_result)
hub_height_slices_data = get_hub_height_slices(real_result)

turbine_layout_tensor_list = get_turbine_layout_tensor()
turbine_infos = turbine_layout_tensor_list[9 - 1]

# Example usage with custom labels, styles, colors, and widths
visualize_speed_profile(
    [hub_height_slices_pred, hub_height_slices_data],
    x_0D_coord=turbine_infos[0, 0].item(),  # Set 0D at x=400
    turbines_info=turbine_infos,  # Turbine info
    plot_y_Range=(600, 2200),  # Plot y range from 600 to 2200 meters
    # plot_neg_D_max_limit = 114514,
    # plot_Ds_max = 114514,
    Diameter=126,
    line_labels=["Ground Truth", "Frozen Hypo"],  # Custom labels
    line_styles=["solid", "dashed"],  # Custom line styles (solid, dashed, dotted)
    line_colors=["black", "#1b60be"],  # Custom colors
    line_widths=[1.5, 2],  # Custom line widths
    save_path="./output/frozen_hypo/profile_t=500.png"  # 保存路径
)


#### inflow visualization

In [ ]:
for i in [0, 160, 164]:
    fig_2 = px.imshow(
        real_result[0, :, :, i].flip(dims=(0,)).cpu().numpy(),
        color_continuous_scale="RdBu_r",
    )
    # 更新布局，隐藏坐标轴和颜色条
    fig_2.update_layout(
        coloraxis_showscale=False,  # 隐藏颜色条
        xaxis=dict(visible=False),  # 隐藏 X 轴
        yaxis=dict(visible=False),  # 隐藏 Y 轴
        paper_bgcolor='rgba(0,0,0,0)',  # 设置图形背景为透明
        plot_bgcolor='rgba(0,0,0,0)',   # 设置绘图区域背景为透明
        width=800,  # 设置图像宽度（可以根据需要调整）
        height=200  # 设置图像高度（可以根据需要调整）
    )
    fig_2.show()

    # 保存逻辑
    save_path = f"./output/inflow_{i}.png"  # 如果不需要保存，将其设为 None
    if save_path:
        # 检查保存路径的目录是否存在，如果不存在则创建
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 导出为透明背景的 PNG
        fig_2.write_image(save_path, format="png")
        print(f"Figure saved to {save_path}")

## FLORIS visualization

In [ ]:
data_floris = realCase_floris_interpolation(
    case_num=6,
    grid_shape=(150, 200),
    is_saving=False,
    save_path=None,
)

fig_2 = px.imshow(
    data_floris[0, :, :],
    color_continuous_scale="RdBu_r",
)

# 更新布局，隐藏坐标轴和颜色条
fig_2.update_layout(
    coloraxis_showscale=False,  # 隐藏颜色条
    xaxis=dict(visible=False),  # 隐藏 X 轴
    yaxis=dict(visible=False),  # 隐藏 Y 轴
    paper_bgcolor='rgba(0,0,0,0)',  # 设置图形背景为透明
    plot_bgcolor='rgba(0,0,0,0)'    # 设置绘图区域背景为透明
)

fig_2.show()

# 添加保存逻辑
save_path = "./output/floris.png"  # 如果不需要保存，将其设为 None

if save_path:
    # 检查路径是否存在，如果不存在则创建
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    # 导出为透明背景的 PNG
    fig_2.write_image(save_path, format="png")
    print(f"Figure saved to {save_path}")

## Effective speed

### effective velocity difference: 2d vs 3d

Because the training set is masked out to 0, you can't view such long range, you can only see the test set from 341 to 400, so this cell's figure will not be the same as the paper's. But the difference is also clear in the test set.

In [ ]:
case_num_list = [9, 6, 10, 11, 12]
paper_case_num_list = [1, 2, 3, 4, 5]
turbine_to_calc_list = [6, 7, 1, 2, 5]

frame_start_idx = 341
frame_end_idx = 400
# Because the training set is masked out to 0, you can't view such long range, 
# you can only see the test set from 341 to 400, so this cell's figure will not be the same as the paper's 
# frame_start_idx = 50
# frame_end_idx = 301

# ############################################################# #
#                load turbine and flow field data               #
# ############################################################# #

for case_num, paper_case_num, turbine_to_calc in zip(case_num_list, paper_case_num_list, turbine_to_calc_list):
    turbine_layout_tensor_list = get_turbine_layout_tensor()
    turbine_layout_tensor = turbine_layout_tensor_list[case_num - 1]
    print(f"case {case_num} turbine to calculate: {turbine_layout_tensor[turbine_to_calc]}")

    data_full_np = np.load(os.path.join(root_path, f"dataset/flowField_full_3d/150_200/adm_{case_num}.npy"))
    data_full = torch.from_numpy(data_full_np).float()
    print(f"data_full.shape: {data_full.shape}")
    # [T=401, C=3, Z=12, H=150, W=200]

    # ############################################################# #
    #                Calculate effective speed at 3d                #
    # ############################################################# #

    effective_speed_frame_list_data = []
    effective_speed_frame_list_pred = []

    for frame in range(frame_start_idx, frame_end_idx):
        data = data_full[frame]

        effective_speed = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], data)
        effective_speed_frame_list_data.append(effective_speed)

        effective_speed = calculate_effective_2d(turbine_layout_tensor[turbine_to_calc], data)
        effective_speed_frame_list_pred.append(effective_speed)

    # print(f"effective_speed_frame_list_3d: {len(effective_speed_frame_list_3d)}")
    # print(f"effective_speed_frame_list_2d: {len(effective_speed_frame_list_2d)}")
    print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_data.__len__()}")
    print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_pred.__len__()}")

    # 创建 x 轴的帧数范围
    frame_range = range(frame_start_idx, frame_end_idx)
    time_range = [i * 10 for i in frame_range]

    # 将 3D 和 2D 的 effective_speed 列表转换为 numpy 数组，以便于绘图
    effective_speed_3d = np.array(effective_speed_frame_list_data)
    effective_speed_2d = np.array(effective_speed_frame_list_pred)

    # 创建一个 matplotlib 图形
    plt.figure(figsize=(16, 4))

    # 绘制 3D effective_speed 的折线图
    plt.plot(time_range[:160], effective_speed_3d[:160], label="Effective Speed 3D", color="black", linestyle="-")

    # 绘制 2D effective_speed 的折线图
    plt.plot(time_range[:160], effective_speed_2d[:160], label="Effective Speed 2D", color="r", linestyle="--")

    # 添加标题和标签
    plt.title(f"Effective Speed for Case {paper_case_num}, Turbine {turbine_to_calc}")
    plt.xlabel("Time")
    plt.ylabel("Effective Speed")

    # 自定义横轴刻度标签为“秒”（后加 's'）
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x)}s"))

    # 显示图例
    plt.legend(fontsize=12)

    # 显示图像
    plt.grid(True)
    plt.tight_layout()

    # 添加保存逻辑
    save_path = (
        f"./output/effective_speed/3d_vs_2d/case_{case_num}_turb_{turbine_to_calc}.png"  # 如果不需要保存，将其设为 None
    )
    if save_path:
        # 检查保存路径是否存在，如果不存在则创建
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        # 保存图片
        plt.savefig(save_path, bbox_inches="tight", dpi=300)  # 使用高分辨率保存
        print(f"Figure saved to {save_path}")

    # 显示图片
    plt.show()

### 3d effective velocity calculation area visualization

In [ ]:
case_num = 9
turbine_to_calc = 6
look_at_frame = 105

# ############################################################# #
#                          turbine info                         #
# ############################################################# #
turbine_layout_tensor_list = get_turbine_layout_tensor()
turbine_layout_tensor = turbine_layout_tensor_list[case_num - 1]
print(f"turbine to calculate: {turbine_layout_tensor[turbine_to_calc]}")

# ############################################################# #
#                          loading data                         #
# ############################################################# #
data_full_np = np.load(os.path.join(root_path, f"dataset/flowField_full_3d/150_200/adm_{case_num}.npy"))
data_full = torch.from_numpy(data_full_np).float()
print(f"data_full.shape: {data_full.shape}")
# [T=401, C=3, Z=12, H=150, W=200]

# ############################################################# #
#                       one frame of data                       #
# ############################################################# #
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
im = ax.imshow(data_full_np[look_at_frame, 0, 4], cmap="RdBu_r")
# 设置 ticks 为空
ax.set_xticks([])
ax.set_yticks([])
# fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# 添加保存逻辑
save_path = "./output/effective_speed/3d_eff_calc_area_vis_full.png"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
fig.savefig(save_path, bbox_inches="tight", dpi=300)
print(f"Horizontal slice saved at {save_path}")

plt.show()

# ############################################################# #
#                          effective_U                          #
# ############################################################# #
# calculate effective U
average_V_eff, mask, upsampled_clip_data_tensor, clip_data_tensor = calculate_effective_3d(
    turbine_info=turbine_layout_tensor[turbine_to_calc], data_tensor=data_full[look_at_frame], debugging_mode=True
)
print(f"average_V_eff: {average_V_eff}")
print(f"mask.shape: {mask.shape}")
# [128, 144, 80]
print(f"upsampled_clip_data_tensor.shape: {upsampled_clip_data_tensor.shape}")
# [3, 128, 272, 336]
print(f"clip_data_tensor.shape: {clip_data_tensor.shape}")
# [3, 8, 17, 21]

mask = mask.float().cpu().numpy()
mask_inverted = 1 - mask
upsampled_clip_data_tensor = upsampled_clip_data_tensor.cpu().numpy()

# horizontal plane slice
fig, ax = plt.subplots(figsize=(8, 6))
im1 = ax.imshow(upsampled_clip_data_tensor[0, 64, :, :], cmap="RdBu_r")
im2 = ax.imshow(mask_inverted[64, :, :], cmap="gray", alpha=0.15)  # alpha=0.5 使 mask 半透明
# 设置 ticks 为空
ax.set_xticks([])
ax.set_yticks([])
# fig.colorbar(im1, ax=ax, fraction=0.046, pad=0.04)
# plt.title("Effective U calc area visualization (cross section)")

# 添加保存逻辑
save_path = "./output/effective_speed/3d_eff_calc_area_vis_hori_plane.png"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
fig.savefig(save_path, bbox_inches="tight", dpi=300)
print(f"Cross section saved at {save_path}")

plt.show()

# vertical plane slice
fig, ax = plt.subplots(figsize=(8, 6))
im1 = ax.imshow(upsampled_clip_data_tensor[0, :, :, 75], cmap="RdBu_r")
im2 = ax.imshow(mask_inverted[:, :, 75], cmap="gray", alpha=0.15)  # alpha=0.5 使 mask 半透明
# 设置 ticks 为空
ax.set_xticks([])
ax.set_yticks([])
# fig.colorbar(im1, ax=ax, fraction=0.046, pad=0.04)
# plt.title("Effective U calc area visualization (vertical profile)")

# 添加保存逻辑
save_path = "./output/effective_speed/3d_eff_calc_area_vis_verti_section.png"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
fig.savefig(save_path, bbox_inches="tight", dpi=300)
print(f"Vertical profile saved at {save_path}")

plt.show()

### 2d effective velocity calculation area visualization

In [ ]:
case_num = 9
turbine_to_calc = 6
look_at_frame = 105

# ############################################################# #
#                          turbine info                         #
# ############################################################# #
turbine_layout_tensor_list = get_turbine_layout_tensor()
turbine_layout_tensor = turbine_layout_tensor_list[case_num - 1]
print(f"turbine to calculate: {turbine_layout_tensor[turbine_to_calc]}")

# ############################################################# #
#                          loading data                         #
# ############################################################# #
data_full_np = np.load(os.path.join(root_path, f"dataset/flowField_full_3d/150_200/adm_{case_num}.npy"))
data_full = torch.from_numpy(data_full_np).float()
print(f"data_full.shape: {data_full.shape}")
# [T=401, C=3, Z=12, H=150, W=200]

# ############################################################# #
#                       one frame of data                       #
# ############################################################# #
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
im = ax.imshow(data_full_np[look_at_frame, 0, 4], cmap="RdBu_r")
fig.colorbar(im)

# 添加保存逻辑
save_path = "./output/effective_speed/2d_eff_calc_area_vis_full.png"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
fig.savefig(save_path, bbox_inches="tight", dpi=300)
print(f"Frame visualization saved at {save_path}")

plt.show()

# ############################################################# #
#                          effective_U                          #
# ############################################################# #
# calculate effective U
average_V_eff, mask, upsampled_clip_data_tensor, clip_data_tensor = calculate_effective_2d(
    turbine_info=turbine_layout_tensor[turbine_to_calc], data_tensor=data_full[look_at_frame], debugging_mode=True
)
print(f"average_V_eff: {average_V_eff}")
print(f"mask.shape: {mask.shape}")
# [128, 144, 80]
print(f"upsampled_clip_data_tensor.shape: {upsampled_clip_data_tensor.shape}")
# [3, 128, 144, 80]
print(f"clip_data_tensor.shape: {clip_data_tensor.shape}")
# [3, 8, 9, 5]

mask = mask.float().cpu().numpy()
mask_inverted = 1 - mask
upsampled_clip_data_tensor = upsampled_clip_data_tensor.cpu().numpy()

fig, ax = plt.subplots(figsize=(8, 6))
im1 = ax.imshow(upsampled_clip_data_tensor[0, :, :], cmap="RdBu_r")
im2 = ax.imshow(mask_inverted[:, :], cmap="gray", alpha=0.1)  # alpha=0.1 使 mask 半透明
fig.colorbar(im1, ax=ax, fraction=0.046, pad=0.04)

# 添加保存逻辑
save_path = "./output/effective_speed/2d_eff_calc_area_hori_plane.png"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
fig.savefig(save_path, bbox_inches="tight", dpi=300)
print(f"Effective U visualization saved at {save_path}")

plt.show()

## data processing fucns

#### get data

In [21]:
def get_single_turbine_space(
    data_tensor: torch.Tensor,
    turbine_info: torch.Tensor,
) -> torch.Tensor:
    """获取单个风机的三维空间的tensor
    截取范围: 风机前40m, 风机后600m, 左右各120m
    对应网格: 风机前2格, 风机后30格,  左右各6格

    Args:
        data_tensor (torch.Tensor): [B,C,Z,H,W], 注意直接读取的数据是 [T,C,Z,H,W], 需要提前处理
        turbine_info (torch.Tensor): [turbine_num, 4] 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]

    Returns:
        torch.Tensor: [turbine_num, C=3, Z=12, H=11, W=33]
        torch.Tensor: [turbine_num, C=3, Z=12, H=11, W=51]
    """
    # 整理index
    indices_no_wake = []
    indices_wake_affected = []
    for turbine in turbine_info:
        x, y, yaw, is_wake_affected = turbine
        space_range = {
            "x_start": x - 2,
            "x_end": x + 31,
            "y_start": y - 6,
            "y_end": y + 7,
        }
        # space_range = {
        #     "x_start": x - 10,
        #     "x_end": x + 40,
        #     "y_start": y - 10,
        #     "y_end": y + 11,
        # }
        # 未受尾流影响的风机尾流预测结果
        if is_wake_affected == 0:
            indices_no_wake.append(space_range)
        # 受尾流影响的风机尾流预测结果
        elif is_wake_affected == 1:
            indices_wake_affected.append(space_range)
        else:
            raise ValueError(f"is_wake_affected should be 0 or 1, but got {is_wake_affected}")

    # 获取数据
    no_wake_data_list = []
    for index in indices_no_wake:
        x_indices = torch.arange(index["x_start"], index["x_end"], dtype=torch.long)
        y_indices = torch.arange(index["y_start"], index["y_end"], dtype=torch.long)
        no_wake_data_list.append(data_tensor[:, :, :, y_indices, :][:, :, :, :, x_indices])
    no_wake_data_tensor = torch.stack(no_wake_data_list, dim=0)

    wake_affected_data_list = []
    for index in indices_wake_affected:
        x_indices = torch.arange(index["x_start"], index["x_end"], dtype=torch.long)
        y_indices = torch.arange(index["y_start"], index["y_end"], dtype=torch.long)
        wake_affected_data_list.append(data_tensor[:, :, :, y_indices, :][:, :, :, :, x_indices])
    wake_affected_data_tensor = torch.stack(wake_affected_data_list, dim=0)

    return no_wake_data_tensor, wake_affected_data_tensor


def load_case_data(case_num: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """从文件夹加载数据，返回torch.Tensor

    Args:
        case_num (int): 1~12

    Returns:
        Tuple[torch.Tensor, torch.Tensor, torch.Tensor]: data_tensor, floris_tensor, turbine_info_tensor

    data_tensor: [T/B=60, C=3, Z=12, H=150, W=200]

    floris_tensor: [C=2, H=150, W=200]

    turbine_info_tensor: [turbine_num, 4] 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]
    
    frozen_flow_tensor: [T=60, C=36, H=150, W=200]
    """
    data_path = os.path.join(root_path, "dataset/flowField_full_3d/150_200", f"adm_{case_num}.npy")
    floris_path = os.path.join(root_path, "dataset/Floris_realCase/150_200", f"adm_{case_num}.npy")
    measured_path = os.path.join(root_path, "dataset/flowField_measured_area_3d/150_200", f"adm_{case_num}.npy")

    # 我们在这里直接完成test集的截取
    # data_tensor: [T=401, C=3, Z=12, H=150, W=200] -> [T=60, C=3, Z=12, H=150, W=200]
    # floris_tensor: [C=2, H=150, W=200]
    # turbine_info_tensor: [turbine_num, 4] 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]
    # measured_tensor: [T=401, C=3, Z=12, H=150, W=200] 
    data_tensor = torch.from_numpy(np.load(data_path)).float()[-60:]
    floris_tensor = torch.from_numpy(np.load(floris_path)).float()
    turbine_layout_tensor = get_turbine_layout_tensor(to_idx=True)
    turbine_info_tensor = turbine_layout_tensor[case_num - 1]
    measured_tensor = torch.from_numpy(np.load(measured_path)).float()[-60-49:]
    # print(f"measured_tensor.shape: {measured_tensor.shape}")

    # 将measured_tensor的数据处理为泰勒冻结流场的数据
    frozen_flow_tensor_list = []
    for i in range(0, 60):
        _tensor = measured_tensor[i:i+50]
        _tensor = torch.flip(_tensor, dims=[0])
        _tensor = einops.rearrange(_tensor, "T C Z H W -> (C Z) H (T W)")
        frozen_flow_tensor_list.append(_tensor)
    frozen_flow_tensor = torch.stack(frozen_flow_tensor_list, dim=0)
    # print(f"frozen_flow_tensor.shape: {frozen_flow_tensor.shape}")
    # frozen_flow_tensor: [T=60, C=36, H=150, W=200]

    return data_tensor, floris_tensor, turbine_info_tensor, frozen_flow_tensor



# -------------------------- 调用方式demo ------------------------- #

# # 指定case_num
# data_tensor, floris_tensor, turbine_info_tensor, frozen_flow_tensor = load_case_data(1)
# print(f"data_tensor.shape: {data_tensor.shape}")
# print(f"floris_tensor.shape: {floris_tensor.shape}")
# print(f"turbine_info_tensor.shape: {turbine_info_tensor.shape}")
# print(f"frozen_flow_tensor.shape: {frozen_flow_tensor.shape}")
# # data_tensor: [T=60, C=3, Z=12, H=150, W=200]
# # floris_tensor: [C=2, H=150, W=200]
# # turbine_info_tensor: [turbine_num, 4] 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]
# # frozen_flow_tensor: [T=60, C=36, H=150, W=200]

# # 单风机切分
# no_wake_data_tensor, wake_affected_data_tensor = get_single_turbine_space(data_tensor, turbine_info_tensor)
# # wake_affected_data_tensor/no_wake_data_tensor: [turbine_num, T/B, C=3, Z=12, H=11, W=33]

# # 绘制单个风机的二维切片，需要三个索引[turbine_num, C, Z]
# plot_single_turbine_space(wake_affected_data_tensor[0, 50, 0, 4])
# plot_single_turbine_space(frozen_flow_tensor[50, 4+12*0])

#### data comparison tool funcs

In [22]:
def compare_pred_target_tensor(pred_tensor, target_tensor, turbine_info_tensor):
    """返回MSE、MAE、RMSE、R^2

    Args:
        pred_tensor (torch.Tensor): NN预测的张量 [B, C, Z, H, W]
        target_tensor (torch.Tensor): 实际张量 [B, C, Z, H, W]
    """
    if pred_tensor.dim() != 5 or target_tensor.dim() != 5:
        console.print(
            f"[bold #918000]Warning![/bold #918000] tensor shape not match. pred: {pred_tensor.shape}, target: {target_tensor.shape}"
        )
        raise ValueError("tensor shape not match.")

    pred_no_wake_data_tensor, pred_wake_affected_data_tensor = get_single_turbine_space(
        pred_tensor, turbine_info_tensor
    )
    target_no_wake_data_tensor, target_wake_affected_data_tensor = get_single_turbine_space(
        target_tensor, turbine_info_tensor
    )

    # 计算 R^2 函数（逐batch维度计算）
    def r2_score_batch(pred, target):
        batch_r2_scores = []
        for i in range(pred.size(0)):  # 遍历 batch 维度
            ss_total = torch.sum((target[i] - target[i].mean()) ** 2)
            ss_residual = torch.sum((target[i] - pred[i]) ** 2)
            r2 = 1 - ss_residual / ss_total if ss_total != 0 else torch.tensor(0.0, device=pred.device)
            batch_r2_scores.append(r2)
        return torch.mean(torch.stack(batch_r2_scores))

    # 计算 no_wake 部分的指标
    MAE_no_wake = F.l1_loss(pred_no_wake_data_tensor, target_no_wake_data_tensor)
    MSE_no_wake = F.mse_loss(pred_no_wake_data_tensor, target_no_wake_data_tensor)
    RMSE_no_wake = torch.sqrt(MSE_no_wake)
    R2_no_wake = r2_score_batch(pred_no_wake_data_tensor, target_no_wake_data_tensor)

    # 计算 wake_affected 部分的指标
    MAE_wake_affected = F.l1_loss(pred_wake_affected_data_tensor, target_wake_affected_data_tensor)
    MSE_wake_affected = F.mse_loss(pred_wake_affected_data_tensor, target_wake_affected_data_tensor)
    RMSE_wake_affected = torch.sqrt(MSE_wake_affected)
    R2_wake_affected = r2_score_batch(pred_wake_affected_data_tensor, target_wake_affected_data_tensor)

    # 计算整体的指标
    MAE_total = F.l1_loss(pred_tensor, target_tensor)
    MSE_total = F.mse_loss(pred_tensor, target_tensor)
    RMSE_total = torch.sqrt(MSE_total)
    R2_total = r2_score_batch(pred_tensor, target_tensor)

    return {
        "no_wake": {
            "MAE": MAE_no_wake,
            "MSE": MSE_no_wake,
            "RMSE": RMSE_no_wake,
            "R2": R2_no_wake,
        },
        "wake_affected": {
            "MAE": MAE_wake_affected,
            "MSE": MSE_wake_affected,
            "RMSE": RMSE_wake_affected,
            "R2": R2_wake_affected,
        },
        "total": {
            "MAE": MAE_total,
            "MSE": MSE_total,
            "RMSE": RMSE_total,
            "R2": R2_total,
        },
    }


def compare_case(case_num: int, pred_func: Callable, print_log: bool = False):
    """对比一个算例的test部分的指标

    Args:
        case_num (int): 1~12
    """
    # 加载数据
    data_tensor, floris_tensor, turbine_info_tensor, frozen_flow_tensor = load_case_data(case_num)
    # data_tensor: [T=60, C=3, Z=12, H=150, W=200]
    # floris_tensor: [C=2, H=150, W=200]
    # turbine_info_tensor: [turbine_num, 4] 4个参数分别是[x网格, y网格, yaw_angle, 前排还是后排]
    # frozen_flow_tensor: [T/B=60, C=36, H=150, W=200]
    min, max = get_min_max_range()

    input_tensor = torch.cat([frozen_flow_tensor, floris_tensor.unsqueeze(0).repeat(frozen_flow_tensor.shape[0], 1, 1, 1)], dim=1)
    input_tensor = min_max_normalize_tensor(input_tensor, min, max)

    # 预测
    input_tensor = input_tensor.to("cuda")
    with torch.no_grad():
        pred_tensor = pred_func(input_tensor)
    pred_tensor = pred_tensor[:, :36]
    pred_tensor = einops.rearrange(pred_tensor, "B (C Z) H W -> B C Z H W", C=3)
    pred_tensor = pred_tensor.cpu()
    pred_tensor = min_max_denormalize_tensor(pred_tensor, min, max)
    
    # 计算指标
    metrics = compare_pred_target_tensor(pred_tensor, data_tensor, turbine_info_tensor)
    if print_log:
        console.print(f"Case {case_num} metrics: {metrics}")

    return metrics, pred_tensor, data_tensor

# min, max = get_min_max_range()
# data_tensor, floris_tensor, turbine_info_tensor, frozen_flow_tensor = load_case_data(12)
# no_wake_data_tensor, wake_affected_data_tensor = get_single_turbine_space(data_tensor, turbine_info_tensor)
# input_tensor = min_max_normalize_tensor(no_wake_data_tensor, min, max)
# back_tensor = min_max_denormalize_tensor(input_tensor, min, max)

# diff = no_wake_data_tensor - back_tensor
# print(f"diff mean: {diff.abs().mean()}")


## Models comparison

From the test below we can see that the turbine-flow interaction is much more complicated than freestram flow.

The test below only considering the turbine wake region and do not consider the freestream.

For detial information about how to clip the region, please see the detail implementation of the comparing function.

In [ ]:
ckpt_path_rectified_flow_autoencoder = f"{root_path}/logs/rectified_flow/lightning_logs/version_69_version_67_wd=2e-6_2/checkpoints/epoch=253-step=15000.ckpt"
ckpt_path_single_autoencoder = f"{root_path}/logs/oneframe_3d/lightning_logs/version_4/checkpoints/epoch=263-step=15000.ckpt"

model_rectified_flow_autoencoder = Rectified_flow.load_from_checkpoint(ckpt_path_rectified_flow_autoencoder)
model_single_autoencoder = Oneframe_Area_lightning.load_from_checkpoint(ckpt_path_single_autoencoder)

In [ ]:
print("\n =================================")
print("currently: diffusion")

metrics = []
pred_tensor_case_list = []
data_tensor_case_list = []

for case_num in [6, 9, 10, 11, 12]:
    metric, pred_tensor, data_tensor = compare_case(
        case_num,
        pred_func=model_rectified_flow_autoencoder.sample_ode_fast,
        print_log=True,  # 打印metrics
    )
    metrics.append(metric)
    pred_tensor_case_list.append(pred_tensor)
    data_tensor_case_list.append(data_tensor)

all_case_pred_tensor = torch.stack(pred_tensor_case_list, dim=0)
all_case_data_tensor = torch.stack(data_tensor_case_list, dim=0)
print(f"all_case_pred_tensor.shape: {all_case_pred_tensor.shape}")
print(f"all_case_data_tensor.shape: {all_case_data_tensor.shape}")
# all_case_pred_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
# all_case_data_tensor: [5, T=60, C=3, Z=12, H=150, W=200]


print("\n =================================")
print("currently: single-step")

metrics_single = []
pred_tensor_case_list_single = []
data_tensor_case_list_single = []

for case_num in [6, 9, 10, 11, 12]:
    metric, pred_tensor, data_tensor = compare_case(
        case_num,
        pred_func=model_single_autoencoder.forward_,
        print_log=True,  # 打印metrics
    )
    metrics_single.append(metric)
    pred_tensor_case_list_single.append(pred_tensor)
    data_tensor_case_list_single.append(data_tensor)

all_case_pred_tensor_single = torch.stack(pred_tensor_case_list_single, dim=0)
all_case_data_tensor_single = torch.stack(data_tensor_case_list_single, dim=0)
print(f"all_case_pred_tensor_single.shape: {all_case_pred_tensor_single.shape}")
print(f"all_case_data_tensor_single.shape: {all_case_data_tensor_single.shape}")

print(f"all_case_data_tensor_single should be same as all_case_data_tensor, diff sum: {torch.sum(all_case_data_tensor_single - all_case_data_tensor)}")

### only diffusion

#### visualization

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor()


case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
turbine_info_idx_list = [9, 6, 10, 11, 12]
timestep_to_view = 50


for selected_case_idx, turbine_infos_idx in zip(case_to_view_list, turbine_info_idx_list):
    print("\n =================================")
    print(f"currently: case {selected_case_idx}")

    case_pred_tensor = all_case_pred_tensor[selected_case_idx]
    case_data_tensor = all_case_data_tensor[selected_case_idx]
    print(f"case_pred_tensor.shape: {case_pred_tensor.shape}")
    print(f"case_data_tensor.shape: {case_data_tensor.shape}")
    # case_pred_tensor: [T=60, C=3, Z=12, H=150, W=200]
    # case_data_tensor: [T=60, C=3, Z=12, H=150, W=200]

    # ----------------------------- 3D ---------------------------- #
    # 绘制 pred 的结果
    plot_3d_slices(
        case_pred_tensor[timestep_to_view],
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Pred",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=750,  # 图形高度
        initial_camera_eye=dict(x=1.8, y=-1.8, z=0.8),  # 从正对面看
        save_path=f"./output/wake_prediction/3d_{turbine_infos_idx}_t={timestep_to_view}_pred.png"
    )

    # 绘制 data 的结果
    plot_3d_slices(
        case_data_tensor[timestep_to_view],
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Target",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=750,  # 图形高度
        initial_camera_eye=dict(x=1.8, y=-1.8, z=0.8),  # 从正对面看
        save_path=f"./output/wake_prediction/3d_{turbine_infos_idx}_t={timestep_to_view}_real.png"
    )

    # ------------------------- 2D slices ------------------------- #
    # 绘制 X 通道的图像（vmin=1, vmax=11）
    fig_x = plot_channels_for_test(
        case_pred_tensor[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=0,
        vmin=1,
        vmax=11,
        channel_name="X",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/wake_prediction/slices_{turbine_infos_idx}_t={timestep_to_view}_x.png"
    )
    fig_x.show()

    # 绘制 Y 通道的图像（vmin=-3, vmax=3）
    fig_y = plot_channels_for_test(
        case_pred_tensor[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=1,
        vmin=-3,
        vmax=3,
        channel_name="Y",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/wake_prediction/slices_{turbine_infos_idx}_t={timestep_to_view}_y.png"
    )
    fig_y.show()

    # 绘制 Z 通道的图像（vmin=-3, vmax=3）
    fig_z = plot_channels_for_test(
        case_pred_tensor[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=2,
        vmin=-3,
        vmax=3,
        channel_name="Z",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/wake_prediction/slices_{turbine_infos_idx}_t={timestep_to_view}_z.png"
    )
    fig_z.show()

    # ------------------ hub height speed profile ----------------- #

    hub_height_slices_pred = get_hub_height_slices(case_pred_tensor[timestep_to_view])
    hub_height_slices_data = get_hub_height_slices(case_data_tensor[timestep_to_view])

    turbine_infos = turbine_layout_tensor_list[turbine_infos_idx - 1]

    visualize_speed_profile(
        [hub_height_slices_pred, hub_height_slices_data],
        x_0D_coord=turbine_infos[0, 0].item(),  # Set 0D at x=400
        turbines_info=turbine_infos,  # Turbine info
        plot_y_Range=(600, 2200),  # Plot y range from 600 to 2200 meters
        Diameter=126,
        line_labels=["Ground Truth", "Prediction"],  # Custom labels
        line_styles=["solid", "dashed"],  # Custom line styles (solid, dashed, dotted)
        line_colors=["black", "#0061ff"],  # Custom colors
        line_widths=[1.5, 2],  # Custom line widths
        save_path=f"./output/wake_prediction/profile_{turbine_infos_idx}_t={timestep_to_view}.png"
    )


#### effective speed comparison

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor()

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
paper_case_num_list = [1, 2, 3, 4, 5]
turbine_layout_case_num_list = [9, 6, 10, 11, 12]
turbines_to_view_list = [
    [1,4,7,8],    # case 9
    [1,4,5,7,8],    # case 6
    [1,6],      # case 10
    [1,2,3],      # case 11
    [1,3,4,5],    # case 12
]

timestep_to_view = 50
frame_start_idx = 0
frame_end_idx = 60

for case_num, paper_case_num, turbine_layout_case_num, case_turbs_to_view_list in zip(case_to_view_list, paper_case_num_list, turbine_layout_case_num_list, turbines_to_view_list):

    for turbine_to_calc in case_turbs_to_view_list:

        print("\n =================================")
        print(f"currently: case {case_num}, 即 case {turbine_layout_case_num}")
        print(f"currently: turbine {turbine_to_calc}")

        # ############################################################# #
        #                load turbine and flow field data               #
        # ############################################################# #

        turbine_layout_tensor = turbine_layout_tensor_list[turbine_layout_case_num - 1]
        print(f"case {turbine_layout_case_num} turbine to calculate: {turbine_layout_tensor[turbine_to_calc]}")

        # all_case_data_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
        # all_case_pred_tensor: [5, T=60, C=3, Z=12, H=150, W=200]

        case_data_tensor = all_case_data_tensor[case_num]
        case_pred_tensor = all_case_pred_tensor[case_num]

        # ############################################################# #
        #                Calculate effective speed at 3d                #
        # ############################################################# #

        effective_speed_frame_list_data = []
        effective_speed_frame_list_pred = []

        for frame in range(frame_start_idx, frame_end_idx):
            data = case_data_tensor[frame]
            pred = case_pred_tensor[frame]

            effective_speed_data = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], data)
            effective_speed_frame_list_data.append(effective_speed_data)

            effective_speed_pred = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], pred)
            effective_speed_frame_list_pred.append(effective_speed_pred)

        print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_data.__len__()}")
        print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_pred.__len__()}")

        # 创建 x 轴的帧数范围
        frame_range = range(frame_start_idx, frame_end_idx)
        time_range = [i * 10 + 10 + 3400 for i in frame_range]

        # 将 3D 和 2D 的 effective_speed 列表转换为 numpy 数组，以便于绘图
        effective_speed_data = np.array(effective_speed_frame_list_data)
        effective_speed_pred = np.array(effective_speed_frame_list_pred)

        # 创建一个 matplotlib 图形
        plt.figure(figsize=(16, 4))

        # 绘制 data effective_speed 的折线图
        plt.plot(time_range[:], effective_speed_data[:], label='Effective Speed Truth', color='black', linestyle='-')

        # 绘制 pred effective_speed 的折线图
        plt.plot(time_range[:], effective_speed_pred[:], label='Effective Speed Pred', color='r', linestyle='--')

        # 添加标题和标签
        plt.title(f'Effective Speed for Case {paper_case_num}, Turbine {turbine_to_calc}')
        plt.xlabel('Time')
        plt.ylabel('Effective Speed')

        # 自定义横轴刻度标签为“秒”（后加 's'）
        ax = plt.gca()
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x)}s"))

        # 显示图例
        plt.legend(fontsize=12)

        # 显示图像
        plt.grid(True)
        plt.tight_layout()

        # 添加保存逻辑
        save_path = f"./output/effective_speed/pred/case_{turbine_layout_case_num}_turb_{turbine_to_calc}.png"  # 如果不需要保存，将其设为 None
        if save_path:
            # 检查保存路径是否存在，如果不存在则创建
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            # 保存图片
            plt.savefig(save_path, bbox_inches="tight", dpi=300)  # 使用高分辨率保存
            print(f"Figure saved to {save_path}")

        plt.show()

#### Time-averaged field comparison

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor(to_idx=True)

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
turbine_layout_case_num_list = [9, 6, 10, 11, 12]

for case_num, turbine_layout_case_num in zip(case_to_view_list, turbine_layout_case_num_list):

    print("\n =================================")
    print(f"currently: case {case_num}")

    # all_case_data_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
    # all_case_pred_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
    # ############################################################# #
    #                       metric calculation                      #
    # ############################################################# #

    turbine_layout_tensor = turbine_layout_tensor_list[turbine_layout_case_num - 1]

    case_data_tensor = all_case_data_tensor[case_num]
    case_pred_tensor = all_case_pred_tensor[case_num]

    averaged_case_data_tensor = case_data_tensor.mean(dim=0).unsqueeze(0)
    averaged_case_pred_tensor = case_pred_tensor.mean(dim=0).unsqueeze(0)
    print(f"averaged_case_data_tensor.shape: {averaged_case_data_tensor.shape}")
    # [1, 3, 12, 150, 200]

    # 计算指标
    metrics = compare_pred_target_tensor(averaged_case_pred_tensor, averaged_case_data_tensor, turbine_layout_tensor)
    console.print(f"metrics of averaged case {turbine_layout_case_num}: {metrics}")

    # ############################################################# #
    #               time-averaged field visualization               #
    # ############################################################# #

    averaged_case_pred_tensor = averaged_case_pred_tensor.squeeze(0)
    averaged_case_data_tensor = averaged_case_data_tensor.squeeze(0)

    # ------------------------- 3D slices ------------------------- #
    # 绘制 pred 的结果
    plot_3d_slices(
        averaged_case_pred_tensor,
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Pred",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=850,  # 图形高度
        initial_camera_eye=dict(x=2.2, y=-1.2, z=2),  # 从正对面看
        save_path=f"./output/average_wake/3d_case_{turbine_layout_case_num}_pred.png"
    )   

    # 绘制 data 的结果
    plot_3d_slices(
        averaged_case_data_tensor,
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Target",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=850,  # 图形高度
        initial_camera_eye=dict(x=2.2, y=-1.2, z=2),  # 从正对面看
        save_path=f"./output/average_wake/3d_case_{turbine_layout_case_num}_data.png"
    )

    # ------------------------- 2D slices ------------------------- #
    # 绘制 X 通道的图像（vmin=1, vmax=11）
    fig_x = plot_channels_for_test(
        averaged_case_pred_tensor,
        averaged_case_data_tensor,
        channel=0,
        vmin=1,
        vmax=11,
        channel_name="X",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/average_wake/slices_case_{turbine_layout_case_num}_x.png"
    )
    fig_x.show()

    # 绘制 Y 通道的图像（vmin=-3, vmax=3）
    fig_y = plot_channels_for_test(
        averaged_case_pred_tensor,
        averaged_case_data_tensor,
        channel=1,
        vmin=-3,
        vmax=3,
        channel_name="Y",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/average_wake/slices_case_{turbine_layout_case_num}_y.png"
    )
    fig_y.show()

    # 绘制 Z 通道的图像（vmin=-3, vmax=3）
    fig_z = plot_channels_for_test(
        averaged_case_pred_tensor,
        averaged_case_data_tensor,
        channel=2,
        vmin=-3,
        vmax=3,
        channel_name="Z",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/average_wake/slices_case_{turbine_layout_case_num}_z.png"
    )
    fig_z.show()

#### 多帧动画导出

In [ ]:
# 假设 all_case_pred_tensor 和 all_case_data_tensor 是给定的张量
# all_case_pred_tensor.shape: [5, 60, 3, 12, 150, 200]
# all_case_data_tensor.shape: [5, 60, 3, 12, 150, 200]

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
case_num_list = [9, 6, 10, 11, 12]

# 动画保存路径
output_dir = "./output/wake_prediction/"
os.makedirs(output_dir, exist_ok=True)

# 动画函数
def create_combined_animation(pred_tensor, data_tensor, case_idx, x_range=(0, 4000), y_range=(0, 3000)):
    """
    创建预测 (Prediction) 和真实数据 (Target) 的对比动画，并保存为 MP4 文件。
    
    参数:
    - pred_tensor: torch.Tensor, shape=[60, 3, 12, 150, 200]
    - data_tensor: torch.Tensor, shape=[60, 3, 12, 150, 200]
    - case_idx: int, 当前 case 的编号
    - x_range: tuple, x 轴的物理范围 (默认: (0, 4000))
    - y_range: tuple, y 轴的物理范围 (默认: (0, 3000))
    """
    # 提取 C=0 (x方向) 和 Z=3 (水平切面)
    pred_data = pred_tensor[:, 0, 3, :, :].cpu().numpy()  # shape: [T=60, H=150, W=200]
    real_data = data_tensor[:, 0, 3, :, :].cpu().numpy()  # shape: [T=60, H=150, W=200]

    # 设置画布
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 两个子图，横向排列
    fig.subplots_adjust(wspace=0.3)  # 调整子图之间的水平间距
    cmap = "RdBu_r"
    vmin, vmax = 1, 11  # 数据范围，用于归一化颜色映射

    # 子图 1: 预测 (Prediction)
    im_pred = axes[0].imshow(pred_data[0], cmap=cmap, vmin=vmin, vmax=vmax, origin="lower",
                              extent=[x_range[0], x_range[1], y_range[0], y_range[1]])
    axes[0].set_title("Prediction", fontsize=14)
    axes[0].set_xlabel("x (meters)", fontsize=12)
    axes[0].set_ylabel("y (meters)", fontsize=12)

    # 子图 2: 真实数据 (Target)
    im_data = axes[1].imshow(real_data[0], cmap=cmap, vmin=vmin, vmax=vmax, origin="lower",
                              extent=[x_range[0], x_range[1], y_range[0], y_range[1]])
    axes[1].set_title("Target", fontsize=14)
    axes[1].set_xlabel("x (meters)", fontsize=12)
    axes[1].set_ylabel("y (meters)", fontsize=12)

    # 全局 colorbar
    cbar = fig.colorbar(im_data, ax=axes, location='right', shrink=0.8, pad=0.05)  # 共享 colorbar，大小适中
    cbar.set_label("Velocity (m/s)", fontsize=12)

    # 更新函数
    def update(frame):
        im_pred.set_data(pred_data[frame])
        im_data.set_data(real_data[frame])
        return [im_pred, im_data]

    # 创建动画
    anim = FuncAnimation(fig, update, frames=60, interval=100, blit=True)

    # 保存动画为 MP4
    save_path = f"{output_dir}case_{case_idx}_comparison.mp4"
    anim.save(save_path, writer="ffmpeg", fps=8)
    plt.close(fig)

    print(f"Animation saved: {save_path}")

# 遍历每个 case 并生成动画
for selected_case_idx, layout_case_num in zip(case_to_view_list, case_num_list):
    print("\n===================================")
    print(f"Currently processing: Case {selected_case_idx}")

    # 提取当前 case 的张量
    case_pred_tensor = all_case_pred_tensor[selected_case_idx]  # [60, 3, 12, 150, 200]
    case_data_tensor = all_case_data_tensor[selected_case_idx]  # [60, 3, 12, 150, 200]

    # 创建对比动画
    create_combined_animation(
        pred_tensor=case_pred_tensor,
        data_tensor=case_data_tensor,
        case_idx=layout_case_num
    )

### diffusion vs single-step

#### visualization

we direct visualize the single-step model slices only

for speed profile, we reuse the hub height slices previewsly computed

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor()

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
turbine_info_idx_list = [9, 6, 10, 11, 12]
timestep_to_view = 50

for selected_case_idx, turbine_infos_idx in zip(case_to_view_list, turbine_info_idx_list):
    print("\n =================================")
    print(f"|Sinlge-Step| currently: case {selected_case_idx}")
    
    case_pred_tensor = all_case_pred_tensor[selected_case_idx]
    case_pred_tensor_single = all_case_pred_tensor_single[selected_case_idx]
    case_data_tensor = all_case_data_tensor[selected_case_idx]
    print(f"case_pred_tensor.shape: {case_pred_tensor.shape}")
    print(f"case_pred_tensor_single.shape: {case_pred_tensor_single.shape}")
    print(f"case_data_tensor.shape: {case_data_tensor.shape}")
    # case_pred_tensor: [T=60, C=3, Z=12, H=150, W=200]
    # case_data_tensor: [T=60, C=3, Z=12, H=150, W=200]

    # ----------------------------- 3D ---------------------------- #
    # 绘制 pred 的结果
    plot_3d_slices(
        case_pred_tensor_single[timestep_to_view],
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Pred",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=750,  # 图形高度
        initial_camera_eye=dict(x=1.8, y=-1.8, z=0.8),  # 从正对面看
        save_path=f"./output/single-step_comparison/3d_{turbine_infos_idx}_t={timestep_to_view}_pred.png"
    )

    # 绘制 data 的结果
    plot_3d_slices(
        case_data_tensor[timestep_to_view],
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Target",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=750,  # 图形高度
        initial_camera_eye=dict(x=1.8, y=-1.8, z=0.8),  # 从正对面看
        save_path=f"./output/single-step_comparison/3d_{turbine_infos_idx}_t={timestep_to_view}_data.png"
    )

    # ------------------------- 2D slices ------------------------- #
    # 绘制 X 通道的图像（vmin=1, vmax=11）
    fig_x = plot_channels_for_test(
        case_pred_tensor_single[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=0,
        vmin=1,
        vmax=11,
        channel_name="X",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/slices_{turbine_infos_idx}_t={timestep_to_view}_x.png"
    )
    fig_x.show()

    # 绘制 Y 通道的图像（vmin=-3, vmax=3）
    fig_y = plot_channels_for_test(
        case_pred_tensor_single[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=1,
        vmin=-3,
        vmax=3,
        channel_name="Y",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/slices_{turbine_infos_idx}_t={timestep_to_view}_y.png"
    )
    fig_y.show()

    # 绘制 Z 通道的图像（vmin=-3, vmax=3）
    fig_z = plot_channels_for_test(
        case_pred_tensor_single[timestep_to_view],
        case_data_tensor[timestep_to_view],
        channel=2,
        vmin=-3,
        vmax=3,
        channel_name="Z",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/slices_{turbine_infos_idx}_t={timestep_to_view}_z.png"
    )
    fig_z.show()

    # ------------------ hub height speed profile ----------------- #

    hub_height_slices_pred = get_hub_height_slices(case_pred_tensor[timestep_to_view])
    hub_height_slices_pred_single = get_hub_height_slices(case_pred_tensor_single[timestep_to_view])
    hub_height_slices_data = get_hub_height_slices(case_data_tensor[timestep_to_view])

    turbine_infos = turbine_layout_tensor_list[turbine_infos_idx - 1]

    visualize_speed_profile(
        [hub_height_slices_pred, hub_height_slices_data, hub_height_slices_pred_single],
        x_0D_coord=turbine_infos[0, 0].item(),  # Set 0D at x=400
        turbines_info=turbine_infos,  # Turbine info
        plot_y_Range=(600, 2200),  # Plot y range from 600 to 2200 meters
        Diameter=126,
        line_labels=["Ground Truth", "Diffusion Pred", "Single-Step Pred"],  # Custom labels
        line_styles=["solid", "dashed", "dashed"],  # Custom line styles (solid, dashed, dotted)
        line_colors=["black", "#0061ff", "red"],  # Custom colors
        line_widths=[1.5, 2, 2],  # Custom line widths
        save_path=f"./output/single-step_comparison/profile_{turbine_infos_idx}_t={timestep_to_view}.png"
    )


#### effective speed comparison

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor()

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
paper_case_num_list = [1, 2, 3, 4, 5]
turbine_layout_case_num_list = [9, 6, 10, 11, 12]
turbines_to_view_list = [
    [1,4,7,8],    # case 9
    [1,4,5,7,8],    # case 6
    [1,6],      # case 10
    [1,2,3],      # case 11
    [1,3,4,5],    # case 12
]

timestep_to_view = 50
frame_start_idx = 0
frame_end_idx = 60

for case_num, paper_case_num, turbine_layout_case_num, case_turbs_to_view_list in zip(case_to_view_list, paper_case_num_list, turbine_layout_case_num_list, turbines_to_view_list):

    for turbine_to_calc in case_turbs_to_view_list:

        print("\n =================================")
        print(f"|Sinlge-Step| currently: case {case_num}")
        print(f"|Sinlge-Step| currently: turbine {turbine_to_calc}")

        # ############################################################# #
        #                load turbine and flow field data               #
        # ############################################################# #

        turbine_layout_tensor = turbine_layout_tensor_list[turbine_layout_case_num - 1]
        print(f"case {turbine_layout_case_num} turbine to calculate: {turbine_layout_tensor[turbine_to_calc]}")

        # all_case_data_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
        # all_case_pred_tensor: [5, T=60, C=3, Z=12, H=150, W=200]

        case_data_tensor = all_case_data_tensor[case_num]
        case_pred_tensor = all_case_pred_tensor[case_num]
        case_pred_tensor_single = all_case_pred_tensor_single[case_num]

        # ############################################################# #
        #                Calculate effective speed at 3d                #
        # ############################################################# #

        effective_speed_frame_list_data = []
        effective_speed_frame_list_pred = []
        effective_speed_frame_list_pred_single = []


        for frame in range(frame_start_idx, frame_end_idx):
            data = case_data_tensor[frame]
            pred = case_pred_tensor[frame]
            pred_single = case_pred_tensor_single[frame]

            effective_speed_data = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], data)
            effective_speed_frame_list_data.append(effective_speed_data)

            effective_speed_pred = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], pred)
            effective_speed_frame_list_pred.append(effective_speed_pred)

            effective_speed_pred_single = calculate_effective_3d(turbine_layout_tensor[turbine_to_calc], pred_single)
            effective_speed_frame_list_pred_single.append(effective_speed_pred_single)

        print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_data.__len__()}")
        print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_pred.__len__()}")
        print(f"effective_speed_frame_list_3d.len: {effective_speed_frame_list_pred_single.__len__()}")

        # 创建 x 轴的帧数范围
        frame_range = range(frame_start_idx, frame_end_idx)
        time_range = [i * 10 + 10 + 3400 for i in frame_range]

        # 将 3D 和 2D 的 effective_speed 列表转换为 numpy 数组，以便于绘图
        effective_speed_data = np.array(effective_speed_frame_list_data)
        effective_speed_pred = np.array(effective_speed_frame_list_pred)
        effective_speed_pred_single = np.array(effective_speed_frame_list_pred_single)

        # 创建一个 matplotlib 图形
        plt.figure(figsize=(16, 4))

        # 绘制 data effective_speed 的折线图
        plt.plot(time_range[:], effective_speed_data[:], label='Effective Speed Truth', color='black', linestyle='-')

        # 绘制 pred effective_speed 的折线图
        plt.plot(time_range[:], effective_speed_pred[:], label='Effective Speed Pred Diffusion', color='r', linestyle='--')

        # 绘制 pred_single effective_speed 的折线图
        plt.plot(time_range[:], effective_speed_pred_single[:], label='Effective Speed Pred Single', color='b', linestyle='--')

        # 添加标题和标签
        plt.title(f'Effective Speed for Case {paper_case_num}, Turbine {turbine_to_calc}')
        plt.xlabel('Time')
        plt.ylabel('Effective Speed')

        # 自定义横轴刻度标签为“秒”（后加 's'）
        ax = plt.gca()
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{int(x)}s"))

        # 显示图例
        plt.legend(fontsize=12)

        # 显示图像
        plt.grid(True)
        plt.tight_layout()

        # 添加保存逻辑
        save_path = f"./output/single-step_comparison/eff_speed_{turbine_layout_case_num}_turb_{turbine_to_calc}.png"  # 如果不需要保存，将其设为 None
        if save_path:
            # 检查保存路径是否存在，如果不存在则创建
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            # 保存图片
            plt.savefig(save_path, bbox_inches="tight", dpi=300)  # 使用高分辨率保存
            print(f"Figure saved to {save_path}")

        plt.show()

#### Time-averaged field comparison

In [ ]:
turbine_layout_tensor_list = get_turbine_layout_tensor(to_idx=True)

case_to_view_list = [Case_idx.case_9.value, Case_idx.case_6.value, Case_idx.case_10.value, Case_idx.case_11.value, Case_idx.case_12.value]
turbine_layout_case_num_list = [9, 6, 10, 11, 12]

for case_num, turbine_layout_case_num in zip(case_to_view_list, turbine_layout_case_num_list):

    print("\n =================================")
    print(f"|Sinlge-Step| currently: case {case_num}")

    # all_case_data_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
    # all_case_pred_tensor: [5, T=60, C=3, Z=12, H=150, W=200]
    # ############################################################# #
    #                       metric calculation                      #
    # ############################################################# #

    turbine_layout_tensor = turbine_layout_tensor_list[turbine_layout_case_num - 1]

    case_data_tensor = all_case_data_tensor[case_num]
    case_pred_tensor = all_case_pred_tensor[case_num]
    case_pred_tensor_single = all_case_pred_tensor_single[case_num]

    averaged_case_data_tensor = case_data_tensor.mean(dim=0).unsqueeze(0)
    averaged_case_pred_tensor = case_pred_tensor.mean(dim=0).unsqueeze(0)
    averaged_case_pred_tensor_single = case_pred_tensor_single.mean(dim=0).unsqueeze(0)

    print(f"averaged_case_data_tensor.shape: {averaged_case_data_tensor.shape}")
    # [1, 3, 12, 150, 200]

    # 计算指标
    metrics = compare_pred_target_tensor(averaged_case_pred_tensor_single, averaged_case_data_tensor, turbine_layout_tensor)
    console.print(f"|Sinlge-Step| metrics of averaged case {turbine_layout_case_num}: {metrics}")

    # ############################################################# #
    #               time-averaged field visualization               #
    # ############################################################# #

    averaged_case_data_tensor = averaged_case_data_tensor.squeeze(0)
    averaged_case_pred_tensor = averaged_case_pred_tensor.squeeze(0)
    averaged_case_pred_tensor_single = averaged_case_pred_tensor_single.squeeze(0)

    # ------------------------- 3D slices ------------------------- #
    # 绘制 pred 的结果
    plot_3d_slices(
        averaged_case_pred_tensor_single,
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Pred",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=850,  # 图形高度
        initial_camera_eye=dict(x=2.2, y=-1.2, z=2),  # 从正对面看
        save_path=f"./output/single-step_comparison/avg_3d_{turbine_layout_case_num}_pred.png"
    )   

    # 绘制 data 的结果
    plot_3d_slices(
        averaged_case_data_tensor,
        C_to_view=0,
        x_slices=[100, 0],
        y_slices=[75, 149],
        z_slices=[0, 6],
        title="Target",
        min_max=(4, 12),
        aspectratio=dict(x=2, y=1.5, z=0.35),
        scale_factor=20,  # 20m一个点
        z_offset=20,  # Z轴从20m开始
        hide_axes=False,  # 隐藏坐标轴
        figure_height=850,  # 图形高度
        initial_camera_eye=dict(x=2.2, y=-1.2, z=2),  # 从正对面看
        save_path=f"./output/single-step_comparison/avg_3d_{turbine_layout_case_num}_data.png"
    )

    # ------------------------- 2D slices ------------------------- #
    # 绘制 X 通道的图像（vmin=1, vmax=11）
    fig_x = plot_channels_for_test(
        averaged_case_pred_tensor_single,
        averaged_case_data_tensor,
        channel=0,
        vmin=1,
        vmax=11,
        channel_name="X",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/avg_slices_{turbine_layout_case_num}_x.png"
    )
    fig_x.show()

    # 绘制 Y 通道的图像（vmin=-3, vmax=3）
    fig_y = plot_channels_for_test(
        averaged_case_pred_tensor_single,
        averaged_case_data_tensor,
        channel=1,
        vmin=-3,
        vmax=3,
        channel_name="Y",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/avg_slices_{turbine_layout_case_num}_y.png"
    )
    fig_y.show()

    # 绘制 Z 通道的图像（vmin=-3, vmax=3）
    fig_z = plot_channels_for_test(
        averaged_case_pred_tensor_single,
        averaged_case_data_tensor,
        channel=2,
        vmin=-3,
        vmax=3,
        channel_name="Z",
        pred_title="Pred",
        target_title="Real",
        save_path=f"./output/single-step_comparison/avg_slices_{turbine_layout_case_num}_z.png"
    )
    fig_z.show()

## Diffusion extra test

Below are some special test for diffusion model & the framework with diffusion model.

This cell will takes very long, if you want to run this part solely, just run the importing and preprocessing, and all visualization functions(this will take within 1 min), and jump here directly.

In [ ]:
model = Rectified_flow.load_from_checkpoint(ckpt_path_rectified_flow_autoencoder)
model.eval()

### Sovling Steps

It takes quite long, so the result is presented here:

'Test/real_MSE_1': 0.1919000744819641,
'Test/real_L1_1': 0.28657811880111694,
'Test/real_RMSE_1': 0.4348478615283966,
'Test/real_R2_1': 0.9878814816474915,

'Test/real_MSE_2': 0.18929389119148254,
'Test/real_L1_2': 0.2831839323043823,
'Test/real_RMSE_2': 0.4319145679473877,
'Test/real_R2_2': 0.988045871257782,

'Test/real_MSE_3': 0.18945461511611938,
'Test/real_L1_3': 0.2829803228378296,
'Test/real_RMSE_3': 0.432115375995636,
'Test/real_R2_3': 0.9880358576774597,

'Test/real_MSE_5': 0.1899195909500122,
'Test/real_L1_5': 0.2831195294857025,
'Test/real_RMSE_5': 0.4326625168323517,
'Test/real_R2_5': 0.9880065321922302,

'Test/real_MSE_10': 0.19045065343379974,
'Test/real_L1_10': 0.28338658809661865,
'Test/real_RMSE_10': 0.43328073620796204,
'Test/real_R2_10': 0.9879729747772217,

'Test/real_MSE_30': 0.19088953733444214,
'Test/real_L1_30': 0.28364190459251404,
'Test/real_RMSE_30': 0.4337889850139618,
'Test/real_R2_30': 0.9879453182220459,

'Test/real_MSE_50': 0.19098524749279022,
'Test/real_L1_50': 0.28370019793510437,
'Test/real_RMSE_50': 0.4338994324207306,
'Test/real_R2_50': 0.987939178943634

In [ ]:
# 加载模型
data_module = Inflow_Area_DataModule()
# 训练配置
trainer = L.Trainer(
    # precision="16-mixed",
    precision="32-true",
    default_root_dir=f"{root_path}/logs/rectified_flow/",
    accelerator="gpu",
)
trainer.test(
    model=model,
    datamodule=data_module,
)


### Time consuming test

#### diffusion model inference time test

need to run two times to fully loading data(because of the virtual memory mechanism in computer system, and we always load all the training and testing dataset)

the performance at first time is not accurate

In [ ]:
# Need to run this cell at least two times to fully load the data and model
# The first time run this cell may need a lot of time loading the whole dataset
# The second time run this cell will be much faster
# When BATCH_SIZE is changed, need to run this cell twice again
BATCH_SIZE = 16


print(f"======= Current Batch Size: {BATCH_SIZE} =======")

class Inspect_Batch_time(Callback):

    def on_predict_batch_start(self, *args, **kwargs):
        """
        在每个批次预测开始时调用的回调，记录批次开始时间。
        """
        # print(f"args.__len__(): {args.__len__()}")
        # print(f"kwargs keys: {kwargs.keys()}")
        # 在 pl_module 中存储开始时间
        self.batch_start_time = time.time()  # 开始计时

    # 结束计时并打印耗时
    def on_predict_batch_end(self, *args, **kwargs):
        """
        在每个批次预测结束时调用的回调，记录批次结束时间并打印耗时。
        """
        elapsed_time = time.time() - self.batch_start_time  # 计算耗时
        print(f"Batch time taken: {elapsed_time:.6f} seconds")


data_module = Inflow_Area_DataModule(
    dataloader_configDict={
        "batch_size": BATCH_SIZE,
        "shuffle": False,
        "num_workers": 0,
        "pin_memory": True,
    }
)
data_module.setup("test")

trainer = L.Trainer(
    precision="16-mixed",
    default_root_dir=f"{root_path}/logs/rectified_flow/",
    accelerator="gpu",
    callbacks=[Inspect_Batch_time()],
)
result = trainer.predict(
    model=model,
    datamodule=data_module,
)

#### Whole framework prediction time test

2080Ti 22G

-------------------
current N: 1
batch_size: 16
Time cost: 9.410047s
num_samples: 300
time cost by each sample: 0.031367s

-------------------
current N: 2
batch_size: 16
Time cost: 20.087921s
num_samples: 300
time cost by each sample: 0.066960s

-------------------
current N: 3
batch_size: 16
Time cost: 30.080214s
num_samples: 300
time cost by each sample: 0.100267s

-------------------
current N: 5
batch_size: 16
Time cost: 50.146573s
num_samples: 300
time cost by each sample: 0.167155s

-------------------
current N: 10
batch_size: 16
Time cost: 100.312163s
num_samples: 300
time cost by each sample: 0.334374s

-------------------
current N: 30
batch_size: 16
Time cost: 300.728757s
num_samples: 300
time cost by each sample: 1.002429s

-------------------
current N: 50
batch_size: 16
Time cost: 500.765674s
num_samples: 300
time cost by each sample: 1.669219s


In [ ]:
class Inflow_Area_Dataset_new(Dataset):
    """3d数据集 适用于多个case给定的不同时间步的数据

    Args:
        input_data_dict_list (list[dict]): 传入的是measured data 宽度上被截取
        floris_data_list (list): 每个case对应一个floris data，因此list长度与input和target相同
        target_data_list (list[dict]): 完整流场数据，没有时间步偏移
        seq_len (int):
        min_val (_type_, optional): Defaults to None.
        max_val (_type_, optional): Defaults to None.

    Raises:
        NotImplementedError: 必须传入min和max值，否则会报错
    """

    def __init__(
        self,
    ) -> None:
        input_data_list = []
        floris_data_list = []
        self.min_val, self.max_val = get_min_max_range()
        self.min_val = torch.tensor(self.min_val, dtype=torch.float32)
        self.max_val = torch.tensor(self.max_val, dtype=torch.float32)
        self.seq_len = 50

        for i in [6, 9, 10, 11, 12]:
            data = np.load(f"{root_path}/dataset/flowField_measured_area_3d/150_200/adm_{i}.npy")
            data = torch.from_numpy(data).to(dtype=torch.float32)
            input_data_list.append(data[-60-50+1:])

        for i in [6, 9, 10, 11, 12]:
            data = np.load(f"{root_path}/dataset/Floris_realCase/150_200/adm_{i}.npy")
            data = torch.from_numpy(data).to(dtype=torch.float32)
            floris_data_list.append(data)

        self.input_data_tensor = torch.stack(input_data_list, dim=0)
        self.floris_data_tensor = torch.stack(floris_data_list, dim=0)
        del input_data_list, floris_data_list
        print(f"self.input_data_tensor.shape: {self.input_data_tensor.shape}")
        print(f"self.floris_data_tensor.shape: {self.floris_data_tensor.shape}")
        # input_data_tensor: (case=3, Timestep=401, C=2, Z=12, H=150, W=8)
        # florisfloris_data_tensor_data: (case=3, C=2, H=150, W=200)

        self.indices = []

        for i, (input_data_case, floris_data_case) in enumerate(zip(self.input_data_tensor, self.floris_data_tensor)):
            for t in range(input_data_case.shape[0] - self.seq_len + 1):
                self.indices.append(
                    (
                        # "case_num":
                        i,
                        # "input_idx":
                        slice(t, t + self.seq_len),
                    )
                )
        
        print(f"len(self.indices): {len(self.indices)}")

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        case_idx, input_idx = self.indices[index]
        input = self.input_data_tensor[case_idx][input_idx]  # [T=50, C=3, Z=12, H=150, W=4]
        floris = self.floris_data_tensor[case_idx]  # [C=2, H, W]

        # Z = [0, 1, 2, 3, 4,  5,  6,  7,  8,  9,  10, 11]
        # Z = [20,40,60,80,100,120,140,160,180,200,220,240]

        # 先flip时间步，然后再连接
        # 如果先连接，那么flip时会把原本的W维度也flip，相当于每个时间步细条都是反着的
        input = torch.flip(input, dims=[0])
        input = einops.rearrange(input, "T C Z H W -> (C Z) H (T W)")

        input = min_max_normalize_tensor(input, self.min_val, self.max_val)

        floris = min_max_normalize_tensor(floris, self.min_val, self.max_val)

        # x0 = torch.cat([input, floris], dim=0)
        # print(f"x0.shape: {x0.shape}")
        # [C=38, H=150, W=200]

        return input, floris

batch_size = 16
this_dataset = Inflow_Area_Dataset_new()
this_dataloader = DataLoader(
    this_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
)
model = Rectified_flow.load_from_checkpoint(ckpt_path_rectified_flow_autoencoder)

# fabric = Fabric(accelerator="cpu")
fabric = Fabric(accelerator="cuda", precision="16-mixed")
model = fabric.setup(model)
this_dataloader = fabric.setup_dataloaders(this_dataloader)


for N in [1, 2, 3, 5, 10, 30, 50]:

    time_counter = time.time()
    model.eval()
    with torch.no_grad():
        for input, floris in this_dataloader:
            x0 = torch.cat([input, floris], dim=1)
            output = model.sample_ode(x0, N=N)[-1]
    time_cost = time.time() - time_counter

    print("-------------------")
    print(f"current N: {N}")
    print(f"batch_size: {batch_size}")
    print(f"Time cost: {time_cost:.6f}s")
    num_samples = this_dataset.__len__()
    print(f"num_samples: {num_samples}")
    print(f"time cost by each sample: {time_cost / num_samples:.6f}s")

# for input, floris, _ in this_dataloader:
#     print(f"input.shape: {input.shape}")
#     print(f"floris.shape: {floris.shape}")